# Preprocessing brain MRI of patients with meningioma

## Before using MONSTR

### Setting PATH variable
**The environment PATH MUST be set in the main terminal, not otherwise like in a sub-terminal in the Jupyterlab**

#### ANTs
export ANTSPATH=/opt/ANTs/bin/

export PATH=${ANTSPATH}:$PATH

#### FSL
FSLDIR=/usr/local/fsl
. ${FSLDIR}/etc/fslconf/fsl.sh

PATH=${FSLDIR}/bin:${PATH}

export FSLDIR PATH


### Update
6/22 \
Code that applies brain mask added.

6/23 \
New variable T1C_label added\
(Sometimes you might want to do co-registration to sequence other than T1C, but this time we assume that you are  registering other sequences to T1C.)\

7/6 \
New variable mask_label added

7/7 \
Add try-except when finding T1Cpath and T2path using glob since T1C and T2 file might not exist

### error
terminate called after throwing an instance of 'std::invalid_argument' 

  what():  stoi

Aborted (core dumped)

==> because of space in filename

### to-do
get pt and seq using re function
add pipeline process would be better

In [1]:
import os
import shutil
import nibabel as nib
import numpy as np
from glob import glob

In [2]:
def make_dirs(path_lst):
    for path in path_lst:
        if not os.path.exists(path):
            os.makedirs(path)
        else:
            pass

# make outputdir and return output path (including filename) by combining outDir, pt, filename
def get_outpath(outDir, pt, filename):
    outPath = os.path.join(outDir, pt, filename)

    make_dirs([os.path.join(outDir, pt)])

    if os.path.exists(outPath):
        print('{} already exists'.format(filename))
        return None
    else:
        return outPath

# find sequence from filename
def find_seq(filename):
    seq_lst = ["T1_GD", "T1GD", "T1CE", "T1C", "CT1", "T1", "T2F", "3D_FLAIR", "FLAIRGD", "FLAIR", "T2", "F", "ADC", "BB", "WB"]
    seq = None
    for i in seq_lst:
        if i in filename.upper():
            seq = i
            return seq
    if seq is None:
        print("No matching sequence")
        return None

In [3]:
sourceDir = '/media/sc-mlsg/easystore/KHS_control_nii_sorted/'
targetDir = '/media/sc-mlsg/easystore/KHS_control_nii_sorted_preprocessed/'

#sourceDir = '/home/sc-mlsg/Downloads/Registry_test48_nii_sorted/'
#targetDir = '/home/sc-mlsg/Downloads/Registry_test48_nii_sorted_preprocessed/'

maskDir = None
outputPath = '/home/scmlsg/MONSTR/output'

resampleDir_mask = os.path.join(targetDir, "00_Tumor_masks_resampled")
#registerDir_mask = os.path.join(targetDir, "00_Tumor_masks_registered")
resampleDir = os.path.join(targetDir, "01_Resampled")
correctionDir = os.path.join(targetDir, "02_Corrected")
registerDir = os.path.join(targetDir, '03_Registered_corrected')
brainmaskDir = os.path.join(targetDir, 'BrainMask_corrected')
strippedDir = os.path.join(targetDir, '04_Stripped_wo_Normalized')
znormedDir = os.path.join(targetDir, '05_Stripped_znormalized')

make_dirs([targetDir, resampleDir, correctionDir, registerDir, brainmaskDir, strippedDir, znormedDir])
make_dirs([resampleDir_mask])

T1C_label = 'BB' # 'BB','T1C','T1GD' etc.
mask_label = 'mask' # '_mask', '-label' etc.

In [11]:
# check directory structure
for (path, dirs, files) in os.walk(sourceDir):
    print(path, dirs, files)

/home/sc-mlsg/Downloads/Registry_test48_nii_sorted/ ['10160204', '10143337', '7395934', '8519617', '8579744', '10110484', '7579610', '10168781', '10208301', '6073063', '10099930', '10158456', '10165247', '10159645', '10064077', '3636668', '8345085', '10054339', '3940812', '10137211', '10182376', '3219808', '10177937', '5652785', '10128196', '8183400', '5810019', '2244819', '10184043', '8316236', '3902261', '2346241', '7135279', '10174199', '10034339', '10204456', '3142491', '10183161', '8091092', '10207110', '10020976', '8543740', '10198829', '7508185'] []
/home/sc-mlsg/Downloads/Registry_test48_nii_sorted/10160204 [] ['10160204_3D_FLAIR.nii.gz', '10160204_FLAIR.nii.gz', '10160204_T2.nii.gz', '10160204_WB.nii.gz', '10160204_T1.nii.gz', '10160204_BB.nii.gz']
/home/sc-mlsg/Downloads/Registry_test48_nii_sorted/10143337 [] ['10143337_FLAIR.nii.gz', '10143337_T1.nii.gz', '10143337_WB.nii.gz', '10143337_3D_FLAIR.nii.gz', '10143337_BB.nii.gz', '10143337_T2.nii.gz']
/home/sc-mlsg/Downloads/Reg

In [6]:
# check file orientation
for (path, dirs, files) in os.walk(sourceDir):
    if files != []:
        for file in files:
            # load img
            img = nib.load(os.path.join(path, file))
            if nib.aff2axcodes(img.affine) != ('L', 'P', 'S'):
                print(file)
                print(nib.aff2axcodes(img.affine))            
            

In [4]:
# 1) resampling
lst = ['postprocessing.json', 'volumes.txt', 'segmentation.nii.gz']
total = 0
for (path, dirs, files) in os.walk(sourceDir):
    if not dirs:
        for file in files:
            if file not in lst:
                total += 1
                print("Resampling: #{}, {}".format(total, file))
                filePath = os.path.join(path, file)
                pt = path.split(os.path.sep)[-1]
                if mask_label in file:
                    pt = path.split(os.path.sep)[-1]
    #                pt = file.split("_")[0]
                    resample_outPath = get_outpath(resampleDir_mask, pt, file)
                else:
                    resample_outPath = get_outpath(resampleDir, pt, file)

                if resample_outPath is None:
                    continue
                !ResampleImage 3 {filePath} {resample_outPath} 1x1x1

Resampling: #1, 10000254_3D_FLAIR.nii.gz
10000254_3D_FLAIR.nii.gz already exists
Resampling: #2, 10000254_BB.nii.gz
10000254_BB.nii.gz already exists
Resampling: #3, 10000254_FLAIR.nii.gz
10000254_FLAIR.nii.gz already exists
Resampling: #4, 10000254_T1.nii.gz
10000254_T1.nii.gz already exists
Resampling: #5, 10000254_T2.nii.gz
10000254_T2.nii.gz already exists
Resampling: #6, 10000254_WB.nii.gz
10000254_WB.nii.gz already exists
Resampling: #7, 10004579_3D_FLAIR.nii.gz
10004579_3D_FLAIR.nii.gz already exists
Resampling: #8, 10004579_BB.nii.gz
10004579_BB.nii.gz already exists
Resampling: #9, 10004579_FLAIR.nii.gz
10004579_FLAIR.nii.gz already exists
Resampling: #10, 10004579_T1.nii.gz
10004579_T1.nii.gz already exists
Resampling: #11, 10004579_T2.nii.gz
10004579_T2.nii.gz already exists
Resampling: #12, 10004579_WB.nii.gz
10004579_WB.nii.gz already exists
Resampling: #13, 10010912_3D_FLAIR.nii.gz
10010912_3D_FLAIR.nii.gz already exists
Resampling: #14, 10010912_BB.nii.gz
10010912_BB.nii

10118423_WB.nii.gz already exists
Resampling: #133, 10136010_3D_FLAIR.nii.gz
10136010_3D_FLAIR.nii.gz already exists
Resampling: #134, 10136010_BB.nii.gz
10136010_BB.nii.gz already exists
Resampling: #135, 10136010_FLAIR.nii.gz
10136010_FLAIR.nii.gz already exists
Resampling: #136, 10136010_T1.nii.gz
10136010_T1.nii.gz already exists
Resampling: #137, 10136010_T2.nii.gz
10136010_T2.nii.gz already exists
Resampling: #138, 10136010_WB.nii.gz
10136010_WB.nii.gz already exists
Resampling: #139, 10139148_3D_FLAIR.nii.gz
10139148_3D_FLAIR.nii.gz already exists
Resampling: #140, 10139148_BB.nii.gz
10139148_BB.nii.gz already exists
Resampling: #141, 10139148_FLAIR.nii.gz
10139148_FLAIR.nii.gz already exists
Resampling: #142, 10139148_T1.nii.gz
10139148_T1.nii.gz already exists
Resampling: #143, 10139148_T2.nii.gz
10139148_T2.nii.gz already exists
Resampling: #144, 10139148_WB.nii.gz
10139148_WB.nii.gz already exists
Resampling: #145, 10139760_3D_FLAIR.nii.gz
10139760_3D_FLAIR.nii.gz already ex

10228949_3D_FLAIR.nii.gz already exists
Resampling: #272, 10228949_BB.nii.gz
10228949_BB.nii.gz already exists
Resampling: #273, 10228949_FLAIR.nii.gz
10228949_FLAIR.nii.gz already exists
Resampling: #274, 10228949_T1.nii.gz
10228949_T1.nii.gz already exists
Resampling: #275, 10228949_T2.nii.gz
10228949_T2.nii.gz already exists
Resampling: #276, 10228949_WB.nii.gz
10228949_WB.nii.gz already exists
Resampling: #277, 10229373_3D_FLAIR.nii.gz
10229373_3D_FLAIR.nii.gz already exists
Resampling: #278, 10229373_BB.nii.gz
10229373_BB.nii.gz already exists
Resampling: #279, 10229373_FLAIR.nii.gz
10229373_FLAIR.nii.gz already exists
Resampling: #280, 10229373_T1.nii.gz
10229373_T1.nii.gz already exists
Resampling: #281, 10229373_T2.nii.gz
10229373_T2.nii.gz already exists
Resampling: #282, 10229373_WB.nii.gz
10229373_WB.nii.gz already exists
Resampling: #283, 10229550_3D_FLAIR.nii.gz
10229550_3D_FLAIR.nii.gz already exists
Resampling: #284, 10229550_BB.nii.gz
10229550_BB.nii.gz already exists
R

8299909_3D_FLAIR.nii.gz already exists
Resampling: #422, 8299909_BB.nii.gz
8299909_BB.nii.gz already exists
Resampling: #423, 8299909_FLAIR.nii.gz
8299909_FLAIR.nii.gz already exists
Resampling: #424, 8299909_T1.nii.gz
8299909_T1.nii.gz already exists
Resampling: #425, 8299909_T2.nii.gz
8299909_T2.nii.gz already exists
Resampling: #426, 8299909_WB.nii.gz
8299909_WB.nii.gz already exists
Resampling: #427, 8309515_3D_FLAIR.nii.gz
8309515_3D_FLAIR.nii.gz already exists
Resampling: #428, 8309515_BB.nii.gz
8309515_BB.nii.gz already exists
Resampling: #429, 8309515_FLAIR.nii.gz
8309515_FLAIR.nii.gz already exists
Resampling: #430, 8309515_T1.nii.gz
8309515_T1.nii.gz already exists
Resampling: #431, 8309515_T2.nii.gz
8309515_T2.nii.gz already exists
Resampling: #432, 8309515_WB.nii.gz
8309515_WB.nii.gz already exists
Resampling: #433, 8344950_3D_FLAIR.nii.gz
8344950_3D_FLAIR.nii.gz already exists
Resampling: #434, 8344950_BB.nii.gz
8344950_BB.nii.gz already exists
Resampling: #435, 8344950_FL

10218119_3D_FLAIR.nii.gz already exists
Resampling: #577, 10218119_BB.nii.gz
10218119_BB.nii.gz already exists
Resampling: #578, 10218119_FLAIR.nii.gz
10218119_FLAIR.nii.gz already exists
Resampling: #579, 10218119_T1.nii.gz
10218119_T1.nii.gz already exists
Resampling: #580, 10218119_T2.nii.gz
10218119_T2.nii.gz already exists
Resampling: #581, 10218119_WB.nii.gz
10218119_WB.nii.gz already exists
Resampling: #582, 10218135_3D_FLAIR.nii.gz
10218135_3D_FLAIR.nii.gz already exists
Resampling: #583, 10218135_BB.nii.gz
10218135_BB.nii.gz already exists
Resampling: #584, 10218135_FLAIR.nii.gz
10218135_FLAIR.nii.gz already exists
Resampling: #585, 10218135_T1.nii.gz
10218135_T1.nii.gz already exists
Resampling: #586, 10218135_T2.nii.gz
10218135_T2.nii.gz already exists
Resampling: #587, 10218135_WB.nii.gz
10218135_WB.nii.gz already exists
Resampling: #588, 10219137_3D_FLAIR.nii.gz
10219137_3D_FLAIR.nii.gz already exists
Resampling: #589, 10219137_BB.nii.gz
10219137_BB.nii.gz already exists
R

...

6866492_WB.nii.gz already exists
Resampling: #984, 6899676_3D_FLAIR.nii.gz
6899676_3D_FLAIR.nii.gz already exists
Resampling: #985, 6899676_BB.nii.gz
6899676_BB.nii.gz already exists
Resampling: #986, 6899676_FLAIR.nii.gz
6899676_FLAIR.nii.gz already exists
Resampling: #987, 6899676_T1.nii.gz
6899676_T1.nii.gz already exists
Resampling: #988, 6899676_T2.nii.gz
6899676_T2.nii.gz already exists
Resampling: #989, 6899676_WB.nii.gz
6899676_WB.nii.gz already exists
Resampling: #990, 7197690_3D_FLAIR.nii.gz
7197690_3D_FLAIR.nii.gz already exists
Resampling: #991, 7197690_BB.nii.gz
7197690_BB.nii.gz already exists
Resampling: #992, 7197690_FLAIR.nii.gz
7197690_FLAIR.nii.gz already exists
Resampling: #993, 7197690_T1.nii.gz
7197690_T1.nii.gz already exists
Resampling: #994, 7197690_T2.nii.gz
7197690_T2.nii.gz already exists
Resampling: #995, 7197690_WB.nii.gz
7197690_WB.nii.gz already exists
Resampling: #996, 7224257_3D_FLAIR.nii.gz
7224257_3D_FLAIR.nii.gz already exists
Resampling: #997, 7224

7966276_BB.nii.gz already exists
Resampling: #1112, 7966276_FLAIR.nii.gz
7966276_FLAIR.nii.gz already exists
Resampling: #1113, 7966276_T1.nii.gz
7966276_T1.nii.gz already exists
Resampling: #1114, 7966276_T2.nii.gz
7966276_T2.nii.gz already exists
Resampling: #1115, 7966276_WB.nii.gz
7966276_WB.nii.gz already exists
Resampling: #1116, 8013591_3D_FLAIR.nii.gz
8013591_3D_FLAIR.nii.gz already exists
Resampling: #1117, 8013591_BB.nii.gz
8013591_BB.nii.gz already exists
Resampling: #1118, 8013591_FLAIR.nii.gz
8013591_FLAIR.nii.gz already exists
Resampling: #1119, 8013591_T1.nii.gz
8013591_T1.nii.gz already exists
Resampling: #1120, 8013591_T2.nii.gz
8013591_T2.nii.gz already exists
Resampling: #1121, 8013591_WB.nii.gz
8013591_WB.nii.gz already exists
Resampling: #1122, 8027498_3D_FLAIR.nii.gz
8027498_3D_FLAIR.nii.gz already exists
Resampling: #1123, 8027498_BB.nii.gz
8027498_BB.nii.gz already exists
Resampling: #1124, 8027498_FLAIR.nii.gz
8027498_FLAIR.nii.gz already exists
Resampling: #11

3321077_T1.nii.gz already exists
Resampling: #1246, 3321077_T2.nii.gz
3321077_T2.nii.gz already exists
Resampling: #1247, 3321077_WB.nii.gz
3321077_WB.nii.gz already exists
Resampling: #1248, 3343416_3D_FLAIR.nii.gz
3343416_3D_FLAIR.nii.gz already exists
Resampling: #1249, 3343416_BB.nii.gz
3343416_BB.nii.gz already exists
Resampling: #1250, 3343416_FLAIR.nii.gz
3343416_FLAIR.nii.gz already exists
Resampling: #1251, 3343416_T1.nii.gz
3343416_T1.nii.gz already exists
Resampling: #1252, 3343416_T2.nii.gz
3343416_T2.nii.gz already exists
Resampling: #1253, 3343416_WB.nii.gz
3343416_WB.nii.gz already exists
Resampling: #1254, 3379783_3D_FLAIR.nii.gz
3379783_3D_FLAIR.nii.gz already exists
Resampling: #1255, 3379783_BB.nii.gz
3379783_BB.nii.gz already exists
Resampling: #1256, 3379783_FLAIR.nii.gz
3379783_FLAIR.nii.gz already exists
Resampling: #1257, 3379783_T1.nii.gz
3379783_T1.nii.gz already exists
Resampling: #1258, 3379783_T2.nii.gz
3379783_T2.nii.gz already exists
Resampling: #1259, 33

10208261_3D_FLAIR.nii.gz already exists
Resampling: #1381, 10208261_BB.nii.gz
10208261_BB.nii.gz already exists
Resampling: #1382, 10208261_FLAIR.nii.gz
10208261_FLAIR.nii.gz already exists
Resampling: #1383, 10208261_T1.nii.gz
10208261_T1.nii.gz already exists
Resampling: #1384, 10208261_T2.nii.gz
10208261_T2.nii.gz already exists
Resampling: #1385, 10208261_WB.nii.gz
10208261_WB.nii.gz already exists
Resampling: #1386, 10210343_3D_FLAIR.nii.gz
10210343_3D_FLAIR.nii.gz already exists
Resampling: #1387, 10210343_BB.nii.gz
10210343_BB.nii.gz already exists
Resampling: #1388, 10210343_FLAIR.nii.gz
10210343_FLAIR.nii.gz already exists
Resampling: #1389, 10210343_T1.nii.gz
10210343_T1.nii.gz already exists
Resampling: #1390, 10210343_T2.nii.gz
10210343_T2.nii.gz already exists
Resampling: #1391, 10210343_WB.nii.gz
10210343_WB.nii.gz already exists
Resampling: #1392, 10210620_3D_FLAIR.nii.gz
10210620_3D_FLAIR.nii.gz already exists
Resampling: #1393, 10210620_BB.nii.gz
10210620_BB.nii.gz alr

8793858_3D_FLAIR.nii.gz already exists
Resampling: #1525, 8793858_BB.nii.gz
8793858_BB.nii.gz already exists
Resampling: #1526, 8793858_FLAIR.nii.gz
8793858_FLAIR.nii.gz already exists
Resampling: #1527, 8793858_T1.nii.gz
8793858_T1.nii.gz already exists
Resampling: #1528, 8793858_T2.nii.gz
8793858_T2.nii.gz already exists
Resampling: #1529, 8793858_WB.nii.gz
8793858_WB.nii.gz already exists
Resampling: #1530, 8794654_3D_FLAIR.nii.gz
8794654_3D_FLAIR.nii.gz already exists
Resampling: #1531, 8794654_BB.nii.gz
8794654_BB.nii.gz already exists
Resampling: #1532, 8794654_FLAIR.nii.gz
8794654_FLAIR.nii.gz already exists
Resampling: #1533, 8794654_T1.nii.gz
8794654_T1.nii.gz already exists
Resampling: #1534, 8794654_T2.nii.gz
8794654_T2.nii.gz already exists
Resampling: #1535, 8794654_WB.nii.gz
8794654_WB.nii.gz already exists
Resampling: #1536, 8799587_3D_FLAIR.nii.gz
8799587_3D_FLAIR.nii.gz already exists
Resampling: #1537, 8799587_BB.nii.gz
8799587_BB.nii.gz already exists
Resampling: #15

In [7]:
# 2) N4 bias correction
total = 0
for (path, dirs, files) in os.walk(resampleDir):
    for file in files:
        total += 1
        print("N4 bias correction: #{}, {}".format(total, file))
        filePath = os.path.join(path, file)
        pt = path.split(os.path.sep)[-1]
        corr_outPath = get_outpath(correctionDir, pt, file)
        if corr_outPath is None:
            continue
        !N4BiasFieldCorrection -d 3 -i {filePath} -o {corr_outPath} -s 3 -c [50x50x50x50,0.00001] -b [150]

N4 bias correction: #1, 10000254_3D_FLAIR.nii.gz
10000254_3D_FLAIR.nii.gz already exists
N4 bias correction: #2, 10000254_BB.nii.gz
10000254_BB.nii.gz already exists
N4 bias correction: #3, 10000254_FLAIR.nii.gz
10000254_FLAIR.nii.gz already exists
N4 bias correction: #4, 10000254_T1.nii.gz
10000254_T1.nii.gz already exists
N4 bias correction: #5, 10000254_T2.nii.gz
10000254_T2.nii.gz already exists
N4 bias correction: #6, 10000254_WB.nii.gz
10000254_WB.nii.gz already exists
N4 bias correction: #7, 10004579_3D_FLAIR.nii.gz
10004579_3D_FLAIR.nii.gz already exists
N4 bias correction: #8, 10004579_BB.nii.gz
10004579_BB.nii.gz already exists
N4 bias correction: #9, 10004579_FLAIR.nii.gz
10004579_FLAIR.nii.gz already exists
N4 bias correction: #10, 10004579_T1.nii.gz
10004579_T1.nii.gz already exists
N4 bias correction: #11, 10004579_T2.nii.gz
10004579_T2.nii.gz already exists
N4 bias correction: #12, 10004579_WB.nii.gz
10004579_WB.nii.gz already exists
N4 bias correction: #13, 10010912_3D_

10139760_3D_FLAIR.nii.gz already exists
N4 bias correction: #146, 10139760_BB.nii.gz
10139760_BB.nii.gz already exists
N4 bias correction: #147, 10139760_FLAIR.nii.gz
10139760_FLAIR.nii.gz already exists
N4 bias correction: #148, 10139760_T1.nii.gz
10139760_T1.nii.gz already exists
N4 bias correction: #149, 10139760_T2.nii.gz
10139760_T2.nii.gz already exists
N4 bias correction: #150, 10139760_WB.nii.gz
10139760_WB.nii.gz already exists
N4 bias correction: #151, 10140324_3D_FLAIR.nii.gz
10140324_3D_FLAIR.nii.gz already exists
N4 bias correction: #152, 10140324_BB.nii.gz
10140324_BB.nii.gz already exists
N4 bias correction: #153, 10140324_FLAIR.nii.gz
10140324_FLAIR.nii.gz already exists
N4 bias correction: #154, 10140324_T1.nii.gz
10140324_T1.nii.gz already exists
N4 bias correction: #155, 10140324_T2.nii.gz
10140324_T2.nii.gz already exists
N4 bias correction: #156, 10140324_WB.nii.gz
10140324_WB.nii.gz already exists
N4 bias correction: #157, 10140649_3D_FLAIR.nii.gz
10140649_3D_FLAI

10207668_3D_FLAIR.nii.gz already exists
N4 bias correction: #296, 10207668_BB.nii.gz
10207668_BB.nii.gz already exists
N4 bias correction: #297, 10207668_FLAIR.nii.gz
10207668_FLAIR.nii.gz already exists
N4 bias correction: #298, 10207668_T1.nii.gz
10207668_T1.nii.gz already exists
N4 bias correction: #299, 10207668_T2.nii.gz
10207668_T2.nii.gz already exists
N4 bias correction: #300, 10207668_WB.nii.gz
10207668_WB.nii.gz already exists
N4 bias correction: #301, 10208261_3D_FLAIR.nii.gz
10208261_3D_FLAIR.nii.gz already exists
N4 bias correction: #302, 10208261_BB.nii.gz
10208261_BB.nii.gz already exists
N4 bias correction: #303, 10208261_FLAIR.nii.gz
10208261_FLAIR.nii.gz already exists
N4 bias correction: #304, 10208261_T1.nii.gz
10208261_T1.nii.gz already exists
N4 bias correction: #305, 10208261_T2.nii.gz
10208261_T2.nii.gz already exists
N4 bias correction: #306, 10208261_WB.nii.gz
10208261_WB.nii.gz already exists
N4 bias correction: #307, 10210343_3D_FLAIR.nii.gz
10210343_3D_FLAI

10219837_3D_FLAIR.nii.gz already exists
N4 bias correction: #410, 10219837_BB.nii.gz
10219837_BB.nii.gz already exists
N4 bias correction: #411, 10219837_FLAIR.nii.gz
10219837_FLAIR.nii.gz already exists
N4 bias correction: #412, 10219837_T1.nii.gz
10219837_T1.nii.gz already exists
N4 bias correction: #413, 10219837_T2.nii.gz
10219837_T2.nii.gz already exists
N4 bias correction: #414, 10219837_WB.nii.gz
10219837_WB.nii.gz already exists
N4 bias correction: #415, 10220063_3D_FLAIR.nii.gz
10220063_3D_FLAIR.nii.gz already exists
N4 bias correction: #416, 10220063_BB.nii.gz
10220063_BB.nii.gz already exists
N4 bias correction: #417, 10220063_FLAIR.nii.gz
10220063_FLAIR.nii.gz already exists
N4 bias correction: #418, 10220063_T1.nii.gz
10220063_T1.nii.gz already exists
N4 bias correction: #419, 10220063_T2.nii.gz
10220063_T2.nii.gz already exists
N4 bias correction: #420, 10220063_WB.nii.gz
10220063_WB.nii.gz already exists
N4 bias correction: #421, 10220403_3D_FLAIR.nii.gz
10220403_3D_FLAI

3014801_3D_FLAIR.nii.gz already exists
N4 bias correction: #518, 3014801_BB.nii.gz
3014801_BB.nii.gz already exists
N4 bias correction: #519, 3014801_FLAIR.nii.gz
3014801_FLAIR.nii.gz already exists
N4 bias correction: #520, 3014801_T1.nii.gz
3014801_T1.nii.gz already exists
N4 bias correction: #521, 3014801_T2.nii.gz
3014801_T2.nii.gz already exists
N4 bias correction: #522, 3014801_WB.nii.gz
3014801_WB.nii.gz already exists
N4 bias correction: #523, 3133622_3D_FLAIR.nii.gz
3133622_3D_FLAIR.nii.gz already exists
N4 bias correction: #524, 3133622_BB.nii.gz
3133622_BB.nii.gz already exists
N4 bias correction: #525, 3133622_FLAIR.nii.gz
3133622_FLAIR.nii.gz already exists
N4 bias correction: #526, 3133622_T1.nii.gz
3133622_T1.nii.gz already exists
N4 bias correction: #527, 3133622_T2.nii.gz
3133622_T2.nii.gz already exists
N4 bias correction: #528, 3133622_WB.nii.gz
3133622_WB.nii.gz already exists
N4 bias correction: #529, 3163796_3D_FLAIR.nii.gz
3163796_3D_FLAIR.nii.gz already exists
N

...

7803649_3D_FLAIR.nii.gz already exists
N4 bias correction: #1081, 7803649_BB.nii.gz
7803649_BB.nii.gz already exists
N4 bias correction: #1082, 7803649_FLAIR.nii.gz
7803649_FLAIR.nii.gz already exists
N4 bias correction: #1083, 7803649_T1.nii.gz
7803649_T1.nii.gz already exists
N4 bias correction: #1084, 7803649_T2.nii.gz
7803649_T2.nii.gz already exists
N4 bias correction: #1085, 7803649_WB.nii.gz
7803649_WB.nii.gz already exists
N4 bias correction: #1086, 7848454_3D_FLAIR.nii.gz
7848454_3D_FLAIR.nii.gz already exists
N4 bias correction: #1087, 7848454_BB.nii.gz
7848454_BB.nii.gz already exists
N4 bias correction: #1088, 7848454_FLAIR.nii.gz
7848454_FLAIR.nii.gz already exists
N4 bias correction: #1089, 7848454_T1.nii.gz
7848454_T1.nii.gz already exists
N4 bias correction: #1090, 7848454_T2.nii.gz
7848454_T2.nii.gz already exists
N4 bias correction: #1091, 7848454_WB.nii.gz
7848454_WB.nii.gz already exists
N4 bias correction: #1092, 7853374_3D_FLAIR.nii.gz
7853374_3D_FLAIR.nii.gz alre

4246570_3D_FLAIR.nii.gz already exists
N4 bias correction: #1219, 4246570_BB.nii.gz
4246570_BB.nii.gz already exists
N4 bias correction: #1220, 4246570_FLAIR.nii.gz
4246570_FLAIR.nii.gz already exists
N4 bias correction: #1221, 4246570_T1.nii.gz
4246570_T1.nii.gz already exists
N4 bias correction: #1222, 4246570_T2.nii.gz
4246570_T2.nii.gz already exists
N4 bias correction: #1223, 4246570_WB.nii.gz
4246570_WB.nii.gz already exists
N4 bias correction: #1224, 4307939_3D_FLAIR.nii.gz
4307939_3D_FLAIR.nii.gz already exists
N4 bias correction: #1225, 4307939_BB.nii.gz
4307939_BB.nii.gz already exists
N4 bias correction: #1226, 4307939_FLAIR.nii.gz
4307939_FLAIR.nii.gz already exists
N4 bias correction: #1227, 4307939_T1.nii.gz
4307939_T1.nii.gz already exists
N4 bias correction: #1228, 4307939_T2.nii.gz
4307939_T2.nii.gz already exists
N4 bias correction: #1229, 4307939_WB.nii.gz
4307939_WB.nii.gz already exists
N4 bias correction: #1230, 4368709_3D_FLAIR.nii.gz
4368709_3D_FLAIR.nii.gz alre

8637081_3D_FLAIR.nii.gz already exists
N4 bias correction: #1363, 8637081_BB.nii.gz
8637081_BB.nii.gz already exists
N4 bias correction: #1364, 8637081_FLAIR.nii.gz
8637081_FLAIR.nii.gz already exists
N4 bias correction: #1365, 8637081_T1.nii.gz
8637081_T1.nii.gz already exists
N4 bias correction: #1366, 8637081_T2.nii.gz
8637081_T2.nii.gz already exists
N4 bias correction: #1367, 8637081_WB.nii.gz
8637081_WB.nii.gz already exists
N4 bias correction: #1368, 8640530_3D_FLAIR.nii.gz
8640530_3D_FLAIR.nii.gz already exists
N4 bias correction: #1369, 8640530_BB.nii.gz
8640530_BB.nii.gz already exists
N4 bias correction: #1370, 8640530_FLAIR.nii.gz
8640530_FLAIR.nii.gz already exists
N4 bias correction: #1371, 8640530_T1.nii.gz
8640530_T1.nii.gz already exists
N4 bias correction: #1372, 8640530_T2.nii.gz
8640530_T2.nii.gz already exists
N4 bias correction: #1373, 8640530_WB.nii.gz
8640530_WB.nii.gz already exists
N4 bias correction: #1374, 8645174_3D_FLAIR.nii.gz
8645174_3D_FLAIR.nii.gz alre

8738230_3D_FLAIR.nii.gz already exists
N4 bias correction: #1465, 8738230_BB.nii.gz
8738230_BB.nii.gz already exists
N4 bias correction: #1466, 8738230_FLAIR.nii.gz
8738230_FLAIR.nii.gz already exists
N4 bias correction: #1467, 8738230_T1.nii.gz
8738230_T1.nii.gz already exists
N4 bias correction: #1468, 8738230_T2.nii.gz
8738230_T2.nii.gz already exists
N4 bias correction: #1469, 8738230_WB.nii.gz
8738230_WB.nii.gz already exists
N4 bias correction: #1470, 8744419_3D_FLAIR.nii.gz
8744419_3D_FLAIR.nii.gz already exists
N4 bias correction: #1471, 8744419_BB.nii.gz
8744419_BB.nii.gz already exists
N4 bias correction: #1472, 8744419_FLAIR.nii.gz
8744419_FLAIR.nii.gz already exists
N4 bias correction: #1473, 8744419_T1.nii.gz
8744419_T1.nii.gz already exists
N4 bias correction: #1474, 8744419_T2.nii.gz
8744419_T2.nii.gz already exists
N4 bias correction: #1475, 8744419_WB.nii.gz
8744419_WB.nii.gz already exists
N4 bias correction: #1476, 8754866_3D_FLAIR.nii.gz
8754866_3D_FLAIR.nii.gz alre

8844889_3D_FLAIR.nii.gz already exists
N4 bias correction: #1567, 8844889_BB.nii.gz
8844889_BB.nii.gz already exists
N4 bias correction: #1568, 8844889_FLAIR.nii.gz
8844889_FLAIR.nii.gz already exists
N4 bias correction: #1569, 8844889_T1.nii.gz
8844889_T1.nii.gz already exists
N4 bias correction: #1570, 8844889_T2.nii.gz
8844889_T2.nii.gz already exists
N4 bias correction: #1571, 8844889_WB.nii.gz
8844889_WB.nii.gz already exists
N4 bias correction: #1572, 8857629_3D_FLAIR.nii.gz
8857629_3D_FLAIR.nii.gz already exists
N4 bias correction: #1573, 8857629_BB.nii.gz
8857629_BB.nii.gz already exists
N4 bias correction: #1574, 8857629_FLAIR.nii.gz
8857629_FLAIR.nii.gz already exists
N4 bias correction: #1575, 8857629_T1.nii.gz
8857629_T1.nii.gz already exists
N4 bias correction: #1576, 8857629_T2.nii.gz
8857629_T2.nii.gz already exists
N4 bias correction: #1577, 8857629_WB.nii.gz
8857629_WB.nii.gz already exists
N4 bias correction: #1578, 8862948_3D_FLAIR.nii.gz
8862948_3D_FLAIR.nii.gz alre

In [8]:
# 3) co-registration
total = 0
for (path, dirs, files) in os.walk(correctionDir):
    if files != []:
        total += 1
        print("coregistration: #{}, {}".format(total, files))
        pt = path.split(os.path.sep)[-1]
        
        try:
            T1C_path = glob(os.path.join(path, '*{}*'.format(T1C_label)))[-1]
        except:
            print("Target sequence not exists")
            continue
        T1C_regist_outPath = get_outpath(registerDir, pt, pt + '_{}_registered.nii'.format(T1C_label))
        if os.path.splitext(T1C_path)[-1] == '.gz':
#            !gunzip T1C_path
            T1C_regist_outPath = get_outpath(registerDir, pt, pt + '_{}_registered.nii.gz'.format(T1C_label))

        if T1C_regist_outPath is not None:
            shutil.copyfile(T1C_path, T1C_regist_outPath)
        
        for file in files:
            seq = find_seq(file)
            if seq == T1C_label:
                continue
            in_path = os.path.join(path, file)
            print("Registering {} to {}...".format(seq, T1C_label))
            outPath = get_outpath(registerDir, pt, pt + '_{}_registered.nii.gz'.format(seq))
            if outPath is None:
                continue

            !antsRegistration -d 3 -r [{T1C_path}, {in_path}, 1] \
                              -m mattes[{T1C_path}, {in_path}, 1, 32, regular, 0.1 ] \
                              -t translation[0.1] -c [100x50x25,1.e-8,20] -s 4x2x1vox -f 6x4x2 -l 1 \
                              -m mattes[{T1C_path}, {in_path} , 1 , 32, regular, 0.1] \
                              -t rigid[0.1] -c [100x50x25,1.e-8,20]  -s 4x2x1vox  -f 3x2x1 -l 1 -o [reg, {outPath}] -v 0


coregistration: #1, ['10000254_3D_FLAIR.nii.gz', '10000254_BB.nii.gz', '10000254_FLAIR.nii.gz', '10000254_T1.nii.gz', '10000254_T2.nii.gz', '10000254_WB.nii.gz']
10000254_BB_registered.nii.gz already exists
Registering 3D_FLAIR to BB...
10000254_3D_FLAIR_registered.nii.gz already exists
Registering FLAIR to BB...
10000254_FLAIR_registered.nii.gz already exists
Registering T1 to BB...
10000254_T1_registered.nii.gz already exists
Registering T2 to BB...
10000254_T2_registered.nii.gz already exists
Registering WB to BB...
10000254_WB_registered.nii.gz already exists
coregistration: #2, ['10004579_3D_FLAIR.nii.gz', '10004579_BB.nii.gz', '10004579_FLAIR.nii.gz', '10004579_T1.nii.gz', '10004579_T2.nii.gz', '10004579_WB.nii.gz']
10004579_BB_registered.nii.gz already exists
Registering 3D_FLAIR to BB...
10004579_3D_FLAIR_registered.nii.gz already exists
Registering FLAIR to BB...
10004579_FLAIR_registered.nii.gz already exists
Registering T1 to BB...
10004579_T1_registered.nii.gz already exist

10088351_BB_registered.nii.gz already exists
Registering 3D_FLAIR to BB...
10088351_3D_FLAIR_registered.nii.gz already exists
Registering FLAIR to BB...
10088351_FLAIR_registered.nii.gz already exists
Registering T1 to BB...
10088351_T1_registered.nii.gz already exists
Registering T2 to BB...
10088351_T2_registered.nii.gz already exists
Registering WB to BB...
10088351_WB_registered.nii.gz already exists
coregistration: #17, ['10097640_3D_FLAIR.nii.gz', '10097640_BB.nii.gz', '10097640_FLAIR.nii.gz', '10097640_T1.nii.gz', '10097640_T2.nii.gz', '10097640_WB.nii.gz']
10097640_BB_registered.nii.gz already exists
Registering 3D_FLAIR to BB...
10097640_3D_FLAIR_registered.nii.gz already exists
Registering FLAIR to BB...
10097640_FLAIR_registered.nii.gz already exists
Registering T1 to BB...
10097640_T1_registered.nii.gz already exists
Registering T2 to BB...
10097640_T2_registered.nii.gz already exists
Registering WB to BB...
10097640_WB_registered.nii.gz already exists
coregistration: #18, 

10151378_FLAIR_registered.nii.gz already exists
Registering T1 to BB...
10151378_T1_registered.nii.gz already exists
Registering T2 to BB...
10151378_T2_registered.nii.gz already exists
Registering WB to BB...
10151378_WB_registered.nii.gz already exists
coregistration: #32, ['10157278_3D_FLAIR.nii.gz', '10157278_BB.nii.gz', '10157278_FLAIR.nii.gz', '10157278_T1.nii.gz', '10157278_T2.nii.gz', '10157278_WB.nii.gz']
10157278_BB_registered.nii.gz already exists
Registering 3D_FLAIR to BB...
10157278_3D_FLAIR_registered.nii.gz already exists
Registering FLAIR to BB...
10157278_FLAIR_registered.nii.gz already exists
Registering T1 to BB...
10157278_T1_registered.nii.gz already exists
Registering T2 to BB...
10157278_T2_registered.nii.gz already exists
Registering WB to BB...
10157278_WB_registered.nii.gz already exists
coregistration: #33, ['10175980_3D_FLAIR.nii.gz', '10175980_BB.nii.gz', '10175980_FLAIR.nii.gz', '10175980_T1.nii.gz', '10175980_T2.nii.gz', '10175980_WB.nii.gz']
10175980_BB

10205378_T1_registered.nii.gz already exists
Registering T2 to BB...
10205378_T2_registered.nii.gz already exists
Registering WB to BB...
10205378_WB_registered.nii.gz already exists
coregistration: #47, ['10205872_3D_FLAIR.nii.gz', '10205872_BB.nii.gz', '10205872_FLAIR.nii.gz', '10205872_T1.nii.gz', '10205872_T2.nii.gz', '10205872_WB.nii.gz']
10205872_BB_registered.nii.gz already exists
Registering 3D_FLAIR to BB...
10205872_3D_FLAIR_registered.nii.gz already exists
Registering FLAIR to BB...
10205872_FLAIR_registered.nii.gz already exists
Registering T1 to BB...
10205872_T1_registered.nii.gz already exists
Registering T2 to BB...
10205872_T2_registered.nii.gz already exists
Registering WB to BB...
10205872_WB_registered.nii.gz already exists
coregistration: #48, ['10206345_3D_FLAIR.nii.gz', '10206345_BB.nii.gz', '10206345_FLAIR.nii.gz', '10206345_T1.nii.gz', '10206345_T2.nii.gz', '10206345_WB.nii.gz']
10206345_BB_registered.nii.gz already exists
Registering 3D_FLAIR to BB...
10206345

10215222_BB_registered.nii.gz already exists
Registering 3D_FLAIR to BB...
10215222_3D_FLAIR_registered.nii.gz already exists
Registering FLAIR to BB...
10215222_FLAIR_registered.nii.gz already exists
Registering T1 to BB...
10215222_T1_registered.nii.gz already exists
Registering T2 to BB...
10215222_T2_registered.nii.gz already exists
Registering WB to BB...
10215222_WB_registered.nii.gz already exists
coregistration: #63, ['10217615_3D_FLAIR.nii.gz', '10217615_BB.nii.gz', '10217615_FLAIR.nii.gz', '10217615_T1.nii.gz', '10217615_T2.nii.gz', '10217615_WB.nii.gz']
10217615_BB_registered.nii.gz already exists
Registering 3D_FLAIR to BB...
10217615_3D_FLAIR_registered.nii.gz already exists
Registering FLAIR to BB...
10217615_FLAIR_registered.nii.gz already exists
Registering T1 to BB...
10217615_T1_registered.nii.gz already exists
Registering T2 to BB...
10217615_T2_registered.nii.gz already exists
Registering WB to BB...
10217615_WB_registered.nii.gz already exists
coregistration: #64, 

...

7664121_BB_registered.nii.gz already exists
Registering 3D_FLAIR to BB...
7664121_3D_FLAIR_registered.nii.gz already exists
Registering FLAIR to BB...
7664121_FLAIR_registered.nii.gz already exists
Registering T1 to BB...
7664121_T1_registered.nii.gz already exists
Registering T2 to BB...
7664121_T2_registered.nii.gz already exists
Registering WB to BB...
7664121_WB_registered.nii.gz already exists
coregistration: #176, ['7678306_3D_FLAIR.nii.gz', '7678306_BB.nii.gz', '7678306_FLAIR.nii.gz', '7678306_T1.nii.gz', '7678306_T2.nii.gz', '7678306_WB.nii.gz']
7678306_BB_registered.nii.gz already exists
Registering 3D_FLAIR to BB...
7678306_3D_FLAIR_registered.nii.gz already exists
Registering FLAIR to BB...
7678306_FLAIR_registered.nii.gz already exists
Registering T1 to BB...
7678306_T1_registered.nii.gz already exists
Registering T2 to BB...
7678306_T2_registered.nii.gz already exists
Registering WB to BB...
7678306_WB_registered.nii.gz already exists
coregistration: #177, ['7715166_3D_FLA

8224066_BB_registered.nii.gz already exists
Registering 3D_FLAIR to BB...
8224066_3D_FLAIR_registered.nii.gz already exists
Registering FLAIR to BB...
8224066_FLAIR_registered.nii.gz already exists
Registering T1 to BB...
8224066_T1_registered.nii.gz already exists
Registering T2 to BB...
8224066_T2_registered.nii.gz already exists
Registering WB to BB...
8224066_WB_registered.nii.gz already exists
coregistration: #195, ['8226738_3D_FLAIR.nii.gz', '8226738_BB.nii.gz', '8226738_FLAIR.nii.gz', '8226738_T1.nii.gz', '8226738_T2.nii.gz', '8226738_WB.nii.gz']
8226738_BB_registered.nii.gz already exists
Registering 3D_FLAIR to BB...
8226738_3D_FLAIR_registered.nii.gz already exists
Registering FLAIR to BB...
8226738_FLAIR_registered.nii.gz already exists
Registering T1 to BB...
8226738_T1_registered.nii.gz already exists
Registering T2 to BB...
8226738_T2_registered.nii.gz already exists
Registering WB to BB...
8226738_WB_registered.nii.gz already exists
coregistration: #196, ['8226987_3D_FLA

5091463_FLAIR_registered.nii.gz already exists
Registering T1 to BB...
5091463_T1_registered.nii.gz already exists
Registering T2 to BB...
5091463_T2_registered.nii.gz already exists
Registering WB to BB...
5091463_WB_registered.nii.gz already exists
coregistration: #211, ['5104858_3D_FLAIR.nii.gz', '5104858_BB.nii.gz', '5104858_FLAIR.nii.gz', '5104858_T1.nii.gz', '5104858_T2.nii.gz', '5104858_WB.nii.gz']
5104858_BB_registered.nii.gz already exists
Registering 3D_FLAIR to BB...
5104858_3D_FLAIR_registered.nii.gz already exists
Registering FLAIR to BB...
5104858_FLAIR_registered.nii.gz already exists
Registering T1 to BB...
5104858_T1_registered.nii.gz already exists
Registering T2 to BB...
5104858_T2_registered.nii.gz already exists
Registering WB to BB...
5104858_WB_registered.nii.gz already exists
coregistration: #212, ['5171633_3D_FLAIR.nii.gz', '5171633_BB.nii.gz', '5171633_FLAIR.nii.gz', '5171633_T1.nii.gz', '5171633_T2.nii.gz', '5171633_WB.nii.gz']
5171633_BB_registered.nii.gz al

8669173_3D_FLAIR_registered.nii.gz already exists
Registering FLAIR to BB...
8669173_FLAIR_registered.nii.gz already exists
Registering T1 to BB...
8669173_T1_registered.nii.gz already exists
Registering T2 to BB...
8669173_T2_registered.nii.gz already exists
Registering WB to BB...
8669173_WB_registered.nii.gz already exists
coregistration: #232, ['8670140_3D_FLAIR.nii.gz', '8670140_BB.nii.gz', '8670140_FLAIR.nii.gz', '8670140_T1.nii.gz', '8670140_T2.nii.gz', '8670140_WB.nii.gz']
8670140_BB_registered.nii.gz already exists
Registering 3D_FLAIR to BB...
8670140_3D_FLAIR_registered.nii.gz already exists
Registering FLAIR to BB...
8670140_FLAIR_registered.nii.gz already exists
Registering T1 to BB...
8670140_T1_registered.nii.gz already exists
Registering T2 to BB...
8670140_T2_registered.nii.gz already exists
Registering WB to BB...
8670140_WB_registered.nii.gz already exists
coregistration: #233, ['8674830_3D_FLAIR.nii.gz', '8674830_BB.nii.gz', '8674830_FLAIR.nii.gz', '8674830_T1.nii.g

8778973_BB_registered.nii.gz already exists
Registering 3D_FLAIR to BB...
8778973_3D_FLAIR_registered.nii.gz already exists
Registering FLAIR to BB...
8778973_FLAIR_registered.nii.gz already exists
Registering T1 to BB...
8778973_T1_registered.nii.gz already exists
Registering T2 to BB...
8778973_T2_registered.nii.gz already exists
Registering WB to BB...
8778973_WB_registered.nii.gz already exists
coregistration: #251, ['8779475_3D_FLAIR.nii.gz', '8779475_BB.nii.gz', '8779475_FLAIR.nii.gz', '8779475_T1.nii.gz', '8779475_T2.nii.gz', '8779475_WB.nii.gz']
8779475_BB_registered.nii.gz already exists
Registering 3D_FLAIR to BB...
8779475_3D_FLAIR_registered.nii.gz already exists
Registering FLAIR to BB...
8779475_FLAIR_registered.nii.gz already exists
Registering T1 to BB...
8779475_T1_registered.nii.gz already exists
Registering T2 to BB...
8779475_T2_registered.nii.gz already exists
Registering WB to BB...
8779475_WB_registered.nii.gz already exists
coregistration: #252, ['8780276_3D_FLA

In [6]:
# 3) co-registration: mask(optional)
total = 0
for (path, dirs, files) in os.walk(resampleDir_mask):
    if files != []:
        total += 1
        print("coregistration: #{}, {}".format(total, files))
        pt = path.split(os.path.sep)[-1]

        try:
            T1C_path = glob(os.path.join(correctionDir, pt, '*{}*'.format(T1C_label)))[0]
        except:
            print("Target sequence not exists")
            continue
        print(T1C_path)
        for file in files:
            in_path = os.path.join(path, file)
            print("Registering mask to {}...".format(T1C_label))
            outPath = get_outpath(registerDir_mask, pt, file)
            if outPath is None:
                continue

            !antsRegistration -d 3 -r [{T1C_path}, {in_path}, 1] \
                              -m mattes[{T1C_path}, {in_path}, 1, 32, regular, 0.1 ] \
                              -t translation[0.1] -c [100x50x25,1.e-8,20] -s 4x2x1vox -f 6x4x2 -l 1 \
                              -m mattes[{T1C_path}, {in_path} , 1 , 32, regular, 0.1] \
                              -t rigid[0.1] -c [100x50x25,1.e-8,20]  -s 4x2x1vox  -f 3x2x1 -l 1 -o [reg, {outPath}] -v 0


coregistration: #1, ['8011358_mask.nii']
8011358
/media/sc-mlsg/Elements/20210322_IDHwtLGG_additional/Previous_patient/Severance_preprocessed/02_Corrected/8011358/8011358_T2F.nii.gz
Registering mask to T2F...
coregistration: #2, ['1371892_mask.nii']
1371892
/media/sc-mlsg/Elements/20210322_IDHwtLGG_additional/Previous_patient/Severance_preprocessed/02_Corrected/1371892/1371892_T2F.nii.gz
Registering mask to T2F...
coregistration: #3, ['2034571_mask.nii']
2034571
/media/sc-mlsg/Elements/20210322_IDHwtLGG_additional/Previous_patient/Severance_preprocessed/02_Corrected/2034571/2034571_T2F.nii.gz
Registering mask to T2F...
coregistration: #4, ['2174093_mask.nii']
2174093
Target sequence not exists
coregistration: #5, ['2545167_mask.nii']
2545167
/media/sc-mlsg/Elements/20210322_IDHwtLGG_additional/Previous_patient/Severance_preprocessed/02_Corrected/2545167/2545167_T2F.nii.gz
Registering mask to T2F...
coregistration: #6, ['2602057_mask.nii']
2602057
/media/sc-mlsg/Elements/20210322_IDHwtL

coregistration: #41, ['8604413_mask.nii']
8604413
/media/sc-mlsg/Elements/20210322_IDHwtLGG_additional/Previous_patient/Severance_preprocessed/02_Corrected/8604413/8604413_T2F.nii.gz
Registering mask to T2F...
coregistration: #42, ['8630035_mask.nii']
8630035
/media/sc-mlsg/Elements/20210322_IDHwtLGG_additional/Previous_patient/Severance_preprocessed/02_Corrected/8630035/8630035_T2F.nii.gz
Registering mask to T2F...
coregistration: #43, ['8633978_mask.nii']
8633978
/media/sc-mlsg/Elements/20210322_IDHwtLGG_additional/Previous_patient/Severance_preprocessed/02_Corrected/8633978/8633978_T2F.nii.gz
Registering mask to T2F...
coregistration: #44, ['8638813_mask.nii']
8638813
/media/sc-mlsg/Elements/20210322_IDHwtLGG_additional/Previous_patient/Severance_preprocessed/02_Corrected/8638813/8638813_T2F.nii.gz
Registering mask to T2F...
coregistration: #45, ['8640861_mask.nii']
8640861
/media/sc-mlsg/Elements/20210322_IDHwtLGG_additional/Previous_patient/Severance_preprocessed/02_Corrected/8640

In [11]:
# 4) Skull stripping using MONSTR

# we'll work in the MONSTR directory
workingDir = '/home/sc-mlsg/MONSTR'
%cd {workingDir} 
outputPath = '/home/sc-mlsg/MONSTR/output'

T1_label = 'T1'
total = 0

for (path, dirs, files) in os.walk(registerDir):
    if files != []:
        total += 1
        print("skull stripping: #{}, {}".format(total, files))

        pt = path.split(os.path.sep)[-1]
        T1_path = glob(os.path.join(path, '*{}*'.format(T1_label)))[0]
        try:
            T2_path = glob(os.path.join(path, '*T2*'))[0]
        except:
            print("T2 not exists")
            T2_path = None
            
        newFileName = pt + '_brain_mask.nii.gz'
        if os.path.exists(os.path.join(brainmaskDir, newFileName)):
            print("{} already exists".format(newFileName))
            continue

        !./MONSTR.sh --t1 {T1_path} --t2 {T2_path} --o output --atlasdir TBI_atlas --reg

        createdDir = os.path.join(outputPath, os.listdir(outputPath)[0])

        for file in os.listdir(createdDir):
            if file == 'atlas4_brainmask_ANTS.nii.gz':
                shutil.copyfile(os.path.join(createdDir, file), os.path.join(brainmaskDir, newFileName))

        shutil.rmtree(createdDir)

/home/sc-mlsg/MONSTR
skull stripping: #1, ['10000254_3D_FLAIR_registered.nii.gz', '10000254_BB_registered.nii.gz', '10000254_FLAIR_registered.nii.gz', '10000254_T1_registered.nii.gz', '10000254_T2_registered.nii.gz', '10000254_WB_registered.nii.gz']
10000254_brain_mask.nii.gz already exists
skull stripping: #2, ['10004579_3D_FLAIR_registered.nii.gz', '10004579_BB_registered.nii.gz', '10004579_FLAIR_registered.nii.gz', '10004579_T1_registered.nii.gz', '10004579_T2_registered.nii.gz', '10004579_WB_registered.nii.gz']
10004579_brain_mask.nii.gz already exists
skull stripping: #3, ['10010912_3D_FLAIR_registered.nii.gz', '10010912_BB_registered.nii.gz', '10010912_FLAIR_registered.nii.gz', '10010912_T1_registered.nii.gz', '10010912_T2_registered.nii.gz', '10010912_WB_registered.nii.gz']
10010912_brain_mask.nii.gz already exists
skull stripping: #4, ['10027303_3D_FLAIR_registered.nii.gz', '10027303_BB_registered.nii.gz', '10027303_FLAIR_registered.nii.gz', '10027303_T1_registered.nii.gz', '10

I found ANTs installation at /opt/ANTs/bin 
Default number of atlases: 4 
Number of processors is not mentioned. Using all available 6 processors. 
Atlas directory : /home/sc-mlsg/MONSTR/TBI_atlas
T1 : /media/sc-mlsg/easystore/KHS_control_nii_sorted_preprocessed/03_Registered_corrected/8814110/8814110_T1_registered.nii.gz
T2 : /media/sc-mlsg/easystore/KHS_control_nii_sorted_preprocessed/03_Registered_corrected/8814110/8814110_T2_registered.nii.gz
Output directory : /home/sc-mlsg/MONSTR/output 
Temporary files will be kept.
Images are assumed to be registered. T2/PD/FLAIR will not be registered again to the T1. 
Working directory : /home/sc-mlsg/MONSTR/output/8814110_T1_registered.nii.xrvj
cd /home/sc-mlsg/MONSTR/output/8814110_T1_registered.nii.xrvj
** AFNI converts NIFTI_datatype=64 (FLOAT64) in file /home/sc-mlsg/MONSTR/output/8814110_T1_registered.nii.xrvj/t1.nii to FLOAT32
     Warnings of this type will be muted for this session.
     Set AFNI_NIFTI_TYPE_WARN to YES to see them al

Output warped image: atlas4_T1_ANTS.nii
ANTS deformable registration took 0 HRS 1 MIN 27 SEC 
Using double precision for computations.
Input scalar image: /home/sc-mlsg/MONSTR/TBI_atlas/atlas4_T2.nii
Reference image: t1.nii
The composite transform comprises the following transforms (in order): 
  1. atlas4_T1_ANTS0GenericAffine.mat (type = AffineTransform)
  2. atlas4_T1_ANTS1Warp.nii.gz (type = DisplacementFieldTransform)
Default pixel value: 0
Interpolation type: BSplineInterpolateImageFunction
Output warped image: atlas4_T2_ANTS.nii
Using double precision for computations.
Input scalar image: /home/sc-mlsg/MONSTR/TBI_atlas/atlas4_brainmask.nii
Reference image: t1.nii
The composite transform comprises the following transforms (in order): 
  1. atlas4_T1_ANTS0GenericAffine.mat (type = AffineTransform)
  2. atlas4_T1_ANTS1Warp.nii.gz (type = DisplacementFieldTransform)
Default pixel value: 0
Interpolation type: NearestNeighborInterpolateImageFunction
Output warped image: atlas4_brainma

atlas2_T1_ANTS.nii:	 16.7% -- replaced with atlas2_T1_ANTS.nii.gz
atlas2_T2_ANTS.nii:	 16.2% -- replaced with atlas2_T2_ANTS.nii.gz
atlas3_brainmask_ANTS.nii:	 99.7% -- replaced with atlas3_brainmask_ANTS.nii.gz
atlas3_T1_ANTS.nii:	 18.4% -- replaced with atlas3_T1_ANTS.nii.gz
atlas3_T2_ANTS.nii:	 17.8% -- replaced with atlas3_T2_ANTS.nii.gz
atlas4_brainmask_ANTS.nii:	 99.7% -- replaced with atlas4_brainmask_ANTS.nii.gz
atlas4_T1_ANTS.nii:	 20.0% -- replaced with atlas4_T1_ANTS.nii.gz
atlas4_T2_ANTS.nii:	 19.5% -- replaced with atlas4_T2_ANTS.nii.gz
t1.nii:	 33.7% -- replaced with t1.nii.gz
t2.nii:	 32.6% -- replaced with t2.nii.gz
Working directory was /home/sc-mlsg/MONSTR/output/8814110_T1_registered.nii.xrvj
MONSTR skull-stripping took 7 MIN 5 SEC
skull stripping: #260, ['8822304_3D_FLAIR_registered.nii.gz', '8822304_BB_registered.nii.gz', '8822304_FLAIR_registered.nii.gz', '8822304_T1_registered.nii.gz', '8822304_T2_registered.nii.gz', '8822304_WB_registered.nii.gz']
I found ANTs i

Output warped image: atlas3_brainmask_ANTS.nii
AntsExample.sh t1.nii /home/sc-mlsg/MONSTR/TBI_atlas/atlas4_T1.nii fastfortesting atlas4_T1_ANTS.nii 6
Using double precision for computations.
Input scalar image: /home/sc-mlsg/MONSTR/TBI_atlas/atlas4_T1.nii
Reference image: t1.nii
The composite transform comprises the following transforms (in order): 
  1. atlas4_T1_ANTS0GenericAffine.mat (type = AffineTransform)
  2. atlas4_T1_ANTS1Warp.nii.gz (type = DisplacementFieldTransform)
Default pixel value: 0
Interpolation type: BSplineInterpolateImageFunction
Output warped image: atlas4_T1_ANTS.nii
ANTS deformable registration took 0 HRS 2 MIN 20 SEC 
Using double precision for computations.
Input scalar image: /home/sc-mlsg/MONSTR/TBI_atlas/atlas4_T2.nii
Reference image: t1.nii
The composite transform comprises the following transforms (in order): 
  1. atlas4_T1_ANTS0GenericAffine.mat (type = AffineTransform)
  2. atlas4_T1_ANTS1Warp.nii.gz (type = DisplacementFieldTransform)
Default pixel v

...

Could not find library, using working directory /home/sc-mlsg/jist-lib
cli	####################################################################
cli	Initializing MIPAV/JIST Framework
Error: Heap size settings in the environment startup file do not match 
those in the Preferences file.
Memory Allocation will display so you can ensure this is correct.
fslmaths t1_MultiModalStripMask_smooth.nii t1_MultiModalStripMask_smooth.nii -odt char 
Image Exception : #63 :: No image files match: t1_MultiModalStripMask_smooth
terminate called after throwing an instance of 'std::runtime_error'
  what():  No image files match: t1_MultiModalStripMask_smooth
./MONSTR.sh: line 578:  8845 Aborted                 (core dumped) fslmaths "$subs"_MultiModalStripMask_smooth.nii "$subs"_MultiModalStripMask_smooth.nii -odt char
/home/sc-mlsg/MONSTR/image_dilate t1_MultiModalStripMask_smooth.nii t1_MultiModalStripMask_smooth.nii 1
/home/sc-mlsg/MONSTR/image_dilate: error while loading shared libraries: libmwlaunche

Reference image: t1.nii
The composite transform comprises the following transforms (in order): 
  1. atlas3_T1_ANTS0GenericAffine.mat (type = AffineTransform)
  2. atlas3_T1_ANTS1Warp.nii.gz (type = DisplacementFieldTransform)
Default pixel value: 0
Interpolation type: BSplineInterpolateImageFunction
Output warped image: atlas3_T1_ANTS.nii
ANTS deformable registration took 0 HRS 1 MIN 29 SEC 
Using double precision for computations.
Input scalar image: /home/sc-mlsg/MONSTR/TBI_atlas/atlas3_T2.nii
Reference image: t1.nii
The composite transform comprises the following transforms (in order): 
  1. atlas3_T1_ANTS0GenericAffine.mat (type = AffineTransform)
  2. atlas3_T1_ANTS1Warp.nii.gz (type = DisplacementFieldTransform)
Default pixel value: 0
Interpolation type: BSplineInterpolateImageFunction
Output warped image: atlas3_T2_ANTS.nii
Using double precision for computations.
Input scalar image: /home/sc-mlsg/MONSTR/TBI_atlas/atlas3_brainmask.nii
Reference image: t1.nii
The composite trans

Could not find library, using working directory /home/sc-mlsg/jist-lib
cli	####################################################################
cli	Initializing MIPAV/JIST Framework
Error: Heap size settings in the environment startup file do not match 
those in the Preferences file.
Memory Allocation will display so you can ensure this is correct.
fslmaths t1_MultiModalStripMask_smooth.nii t1_MultiModalStripMask_smooth.nii -odt char 
Image Exception : #63 :: No image files match: t1_MultiModalStripMask_smooth
terminate called after throwing an instance of 'std::runtime_error'
  what():  No image files match: t1_MultiModalStripMask_smooth
./MONSTR.sh: line 578:  9306 Aborted                 (core dumped) fslmaths "$subs"_MultiModalStripMask_smooth.nii "$subs"_MultiModalStripMask_smooth.nii -odt char
/home/sc-mlsg/MONSTR/image_dilate t1_MultiModalStripMask_smooth.nii t1_MultiModalStripMask_smooth.nii 1
/home/sc-mlsg/MONSTR/image_dilate: error while loading shared libraries: libmwlaunche

Reference image: t1.nii
The composite transform comprises the following transforms (in order): 
  1. atlas3_T1_ANTS0GenericAffine.mat (type = AffineTransform)
  2. atlas3_T1_ANTS1Warp.nii.gz (type = DisplacementFieldTransform)
Default pixel value: 0
Interpolation type: BSplineInterpolateImageFunction
Output warped image: atlas3_T1_ANTS.nii
ANTS deformable registration took 0 HRS 1 MIN 25 SEC 
Using double precision for computations.
Input scalar image: /home/sc-mlsg/MONSTR/TBI_atlas/atlas3_T2.nii
Reference image: t1.nii
The composite transform comprises the following transforms (in order): 
  1. atlas3_T1_ANTS0GenericAffine.mat (type = AffineTransform)
  2. atlas3_T1_ANTS1Warp.nii.gz (type = DisplacementFieldTransform)
Default pixel value: 0
Interpolation type: BSplineInterpolateImageFunction
Output warped image: atlas3_T2_ANTS.nii
Using double precision for computations.
Input scalar image: /home/sc-mlsg/MONSTR/TBI_atlas/atlas3_brainmask.nii
Reference image: t1.nii
The composite trans

Could not find library, using working directory /home/sc-mlsg/jist-lib
cli	####################################################################
cli	Initializing MIPAV/JIST Framework
Error: Heap size settings in the environment startup file do not match 
those in the Preferences file.
Memory Allocation will display so you can ensure this is correct.
fslmaths t1_MultiModalStripMask_smooth.nii t1_MultiModalStripMask_smooth.nii -odt char 
Image Exception : #63 :: No image files match: t1_MultiModalStripMask_smooth
terminate called after throwing an instance of 'std::runtime_error'
  what():  No image files match: t1_MultiModalStripMask_smooth
./MONSTR.sh: line 578:  9720 Aborted                 (core dumped) fslmaths "$subs"_MultiModalStripMask_smooth.nii "$subs"_MultiModalStripMask_smooth.nii -odt char
/home/sc-mlsg/MONSTR/image_dilate t1_MultiModalStripMask_smooth.nii t1_MultiModalStripMask_smooth.nii 1
/home/sc-mlsg/MONSTR/image_dilate: error while loading shared libraries: libmwlaunche

In [12]:
# 5) apply skull mask
total = 0
error_dict = {}
for (path, dirs, files) in os.walk(registerDir):
    try:
        if files != []:
            total += 1
            pt = path.split(os.path.sep)[-1]
            maskName = pt + '_brain_mask.nii.gz'
            mask = nib.load(os.path.join(brainmaskDir, maskName))

            for file in files:
                seq = find_seq(file)
                print("# {} Applying skull mask to {}...".format(total, file))
                outPath = get_outpath(strippedDir, pt, pt + "_{}_stripped.nii.gz".format(seq))
                if outPath is None:
                    continue
                
                # Reshape (in some cases data resolution is (x,y,z,1), reshape removes the 4th dimension
                file_nii = nib.load(os.path.join(path, file))
                file_data = file_nii.get_data()
                file_data = file_data.reshape(file_data.shape[0], file_data.shape[1], file_data.shape[2])

                masked_data = np.multiply(file_data, mask.get_data())
                masked_data = nib.Nifti1Image(masked_data, file_nii.affine, file_nii.header)
                nib.save(masked_data, outPath)
    except Exception as ex:
        print("Error:{}, filename:{}".format(ex.args, file))
        error_dict[pt] = ex.args

print("Finished!")
print(error_dict)

# 1 Applying skull mask to 10000254_3D_FLAIR_registered.nii.gz...
10000254_3D_FLAIR_stripped.nii.gz already exists
# 1 Applying skull mask to 10000254_BB_registered.nii.gz...
10000254_BB_stripped.nii.gz already exists
# 1 Applying skull mask to 10000254_FLAIR_registered.nii.gz...
10000254_FLAIR_stripped.nii.gz already exists
# 1 Applying skull mask to 10000254_T1_registered.nii.gz...
10000254_T1_stripped.nii.gz already exists
# 1 Applying skull mask to 10000254_T2_registered.nii.gz...
10000254_T2_stripped.nii.gz already exists
# 1 Applying skull mask to 10000254_WB_registered.nii.gz...
10000254_WB_stripped.nii.gz already exists
# 2 Applying skull mask to 10004579_3D_FLAIR_registered.nii.gz...
10004579_3D_FLAIR_stripped.nii.gz already exists
# 2 Applying skull mask to 10004579_BB_registered.nii.gz...
10004579_BB_stripped.nii.gz already exists
# 2 Applying skull mask to 10004579_FLAIR_registered.nii.gz...
10004579_FLAIR_stripped.nii.gz already exists
# 2 Applying skull mask to 10004579_T

10175980_T2_stripped.nii.gz already exists
# 33 Applying skull mask to 10175980_WB_registered.nii.gz...
10175980_WB_stripped.nii.gz already exists
# 34 Applying skull mask to 10177293_3D_FLAIR_registered.nii.gz...
10177293_3D_FLAIR_stripped.nii.gz already exists
# 34 Applying skull mask to 10177293_BB_registered.nii.gz...
10177293_BB_stripped.nii.gz already exists
# 34 Applying skull mask to 10177293_FLAIR_registered.nii.gz...
10177293_FLAIR_stripped.nii.gz already exists
# 34 Applying skull mask to 10177293_T1_registered.nii.gz...
10177293_T1_stripped.nii.gz already exists
# 34 Applying skull mask to 10177293_T2_registered.nii.gz...
10177293_T2_stripped.nii.gz already exists
# 34 Applying skull mask to 10177293_WB_registered.nii.gz...
10177293_WB_stripped.nii.gz already exists
# 35 Applying skull mask to 10180370_3D_FLAIR_registered.nii.gz...
10180370_3D_FLAIR_stripped.nii.gz already exists
# 35 Applying skull mask to 10180370_BB_registered.nii.gz...
10180370_BB_stripped.nii.gz alread

# 66 Applying skull mask to 10218135_3D_FLAIR_registered.nii.gz...
10218135_3D_FLAIR_stripped.nii.gz already exists
# 66 Applying skull mask to 10218135_BB_registered.nii.gz...
10218135_BB_stripped.nii.gz already exists
# 66 Applying skull mask to 10218135_FLAIR_registered.nii.gz...
10218135_FLAIR_stripped.nii.gz already exists
# 66 Applying skull mask to 10218135_T1_registered.nii.gz...
10218135_T1_stripped.nii.gz already exists
# 66 Applying skull mask to 10218135_T2_registered.nii.gz...
10218135_T2_stripped.nii.gz already exists
# 66 Applying skull mask to 10218135_WB_registered.nii.gz...
10218135_WB_stripped.nii.gz already exists
# 67 Applying skull mask to 10219137_3D_FLAIR_registered.nii.gz...
10219137_3D_FLAIR_stripped.nii.gz already exists
# 67 Applying skull mask to 10219137_BB_registered.nii.gz...
10219137_BB_stripped.nii.gz already exists
# 67 Applying skull mask to 10219137_FLAIR_registered.nii.gz...
10219137_FLAIR_stripped.nii.gz already exists
# 67 Applying skull mask to 

# 100 Applying skull mask to 3133622_3D_FLAIR_registered.nii.gz...
3133622_3D_FLAIR_stripped.nii.gz already exists
# 100 Applying skull mask to 3133622_BB_registered.nii.gz...
3133622_BB_stripped.nii.gz already exists
# 100 Applying skull mask to 3133622_FLAIR_registered.nii.gz...
3133622_FLAIR_stripped.nii.gz already exists
# 100 Applying skull mask to 3133622_T1_registered.nii.gz...
3133622_T1_stripped.nii.gz already exists
# 100 Applying skull mask to 3133622_T2_registered.nii.gz...
3133622_T2_stripped.nii.gz already exists
# 100 Applying skull mask to 3133622_WB_registered.nii.gz...
3133622_WB_stripped.nii.gz already exists
# 101 Applying skull mask to 3163796_3D_FLAIR_registered.nii.gz...
3163796_3D_FLAIR_stripped.nii.gz already exists
# 101 Applying skull mask to 3163796_BB_registered.nii.gz...
3163796_BB_stripped.nii.gz already exists
# 101 Applying skull mask to 3163796_FLAIR_registered.nii.gz...
3163796_FLAIR_stripped.nii.gz already exists
# 101 Applying skull mask to 3163796_

2081039_3D_FLAIR_stripped.nii.gz already exists
# 131 Applying skull mask to 2081039_BB_registered.nii.gz...
2081039_BB_stripped.nii.gz already exists
# 131 Applying skull mask to 2081039_FLAIR_registered.nii.gz...
2081039_FLAIR_stripped.nii.gz already exists
# 131 Applying skull mask to 2081039_T1_registered.nii.gz...
2081039_T1_stripped.nii.gz already exists
# 131 Applying skull mask to 2081039_T2_registered.nii.gz...
2081039_T2_stripped.nii.gz already exists
# 131 Applying skull mask to 2081039_WB_registered.nii.gz...
2081039_WB_stripped.nii.gz already exists
# 132 Applying skull mask to 2125264_3D_FLAIR_registered.nii.gz...
2125264_3D_FLAIR_stripped.nii.gz already exists
# 132 Applying skull mask to 2125264_BB_registered.nii.gz...
2125264_BB_stripped.nii.gz already exists
# 132 Applying skull mask to 2125264_FLAIR_registered.nii.gz...
2125264_FLAIR_stripped.nii.gz already exists
# 132 Applying skull mask to 2125264_T1_registered.nii.gz...
2125264_T1_stripped.nii.gz already exists
#

7803649_3D_FLAIR_stripped.nii.gz already exists
# 162 Applying skull mask to 7803649_BB_registered.nii.gz...
7803649_BB_stripped.nii.gz already exists
# 162 Applying skull mask to 7803649_FLAIR_registered.nii.gz...
7803649_FLAIR_stripped.nii.gz already exists
# 162 Applying skull mask to 7803649_T1_registered.nii.gz...
7803649_T1_stripped.nii.gz already exists
# 162 Applying skull mask to 7803649_T2_registered.nii.gz...
7803649_T2_stripped.nii.gz already exists
# 162 Applying skull mask to 7803649_WB_registered.nii.gz...
7803649_WB_stripped.nii.gz already exists
# 163 Applying skull mask to 7848454_3D_FLAIR_registered.nii.gz...
7848454_3D_FLAIR_stripped.nii.gz already exists
# 163 Applying skull mask to 7848454_BB_registered.nii.gz...
7848454_BB_stripped.nii.gz already exists
# 163 Applying skull mask to 7848454_FLAIR_registered.nii.gz...
7848454_FLAIR_stripped.nii.gz already exists
# 163 Applying skull mask to 7848454_T1_registered.nii.gz...
7848454_T1_stripped.nii.gz already exists
#

# 195 Applying skull mask to 8577689_3D_FLAIR_registered.nii.gz...
8577689_3D_FLAIR_stripped.nii.gz already exists
# 195 Applying skull mask to 8577689_BB_registered.nii.gz...
8577689_BB_stripped.nii.gz already exists
# 195 Applying skull mask to 8577689_FLAIR_registered.nii.gz...
8577689_FLAIR_stripped.nii.gz already exists
# 195 Applying skull mask to 8577689_T1_registered.nii.gz...
8577689_T1_stripped.nii.gz already exists
# 195 Applying skull mask to 8577689_T2_registered.nii.gz...
8577689_T2_stripped.nii.gz already exists
# 195 Applying skull mask to 8577689_WB_registered.nii.gz...
8577689_WB_stripped.nii.gz already exists
# 196 Applying skull mask to 8583970_3D_FLAIR_registered.nii.gz...
8583970_3D_FLAIR_stripped.nii.gz already exists
# 196 Applying skull mask to 8583970_BB_registered.nii.gz...
8583970_BB_stripped.nii.gz already exists
# 196 Applying skull mask to 8583970_FLAIR_registered.nii.gz...
8583970_FLAIR_stripped.nii.gz already exists
# 196 Applying skull mask to 8583970_

# 228 Applying skull mask to 8645174_3D_FLAIR_registered.nii.gz...
8645174_3D_FLAIR_stripped.nii.gz already exists
# 228 Applying skull mask to 8645174_BB_registered.nii.gz...
8645174_BB_stripped.nii.gz already exists
# 228 Applying skull mask to 8645174_FLAIR_registered.nii.gz...
8645174_FLAIR_stripped.nii.gz already exists
# 228 Applying skull mask to 8645174_T1_registered.nii.gz...
8645174_T1_stripped.nii.gz already exists
# 228 Applying skull mask to 8645174_T2_registered.nii.gz...
8645174_T2_stripped.nii.gz already exists
# 228 Applying skull mask to 8645174_WB_registered.nii.gz...
8645174_WB_stripped.nii.gz already exists
# 229 Applying skull mask to 8658118_3D_FLAIR_registered.nii.gz...
8658118_3D_FLAIR_stripped.nii.gz already exists
# 229 Applying skull mask to 8658118_BB_registered.nii.gz...
8658118_BB_stripped.nii.gz already exists
# 229 Applying skull mask to 8658118_FLAIR_registered.nii.gz...
8658118_FLAIR_stripped.nii.gz already exists
# 229 Applying skull mask to 8658118_

/home/sc-mlsg/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:21: DeprecationWarning: get_data() is deprecated in favor of get_fdata(), which has a more predictable return type. To obtain get_data() behavior going forward, use numpy.asanyarray(img.dataobj).

* deprecated from version: 3.0
* Will raise <class 'nibabel.deprecator.ExpiredDeprecationError'> as of version: 5.0
/home/sc-mlsg/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:24: DeprecationWarning: get_data() is deprecated in favor of get_fdata(), which has a more predictable return type. To obtain get_data() behavior going forward, use numpy.asanyarray(img.dataobj).

* deprecated from version: 3.0
* Will raise <class 'nibabel.deprecator.ExpiredDeprecationError'> as of version: 5.0


# 258 Applying skull mask to 8814110_BB_registered.nii.gz...
# 258 Applying skull mask to 8814110_FLAIR_registered.nii.gz...
# 258 Applying skull mask to 8814110_T1_registered.nii.gz...
# 258 Applying skull mask to 8814110_T2_registered.nii.gz...
# 258 Applying skull mask to 8814110_WB_registered.nii.gz...
# 259 Applying skull mask to 8822304_3D_FLAIR_registered.nii.gz...
# 259 Applying skull mask to 8822304_BB_registered.nii.gz...
# 259 Applying skull mask to 8822304_FLAIR_registered.nii.gz...
# 259 Applying skull mask to 8822304_T1_registered.nii.gz...
# 259 Applying skull mask to 8822304_T2_registered.nii.gz...
# 259 Applying skull mask to 8822304_WB_registered.nii.gz...
# 260 Applying skull mask to 8824792_3D_FLAIR_registered.nii.gz...
# 260 Applying skull mask to 8824792_BB_registered.nii.gz...
# 260 Applying skull mask to 8824792_FLAIR_registered.nii.gz...
# 260 Applying skull mask to 8824792_T1_registered.nii.gz...
# 260 Applying skull mask to 8824792_T2_registered.nii.gz...
# 2

In [12]:
# 6) z-score normalization
total = 0
for (path, dirs, files) in os.walk(registerDir):
    for file in files:
        try:
            total += 1
            print("Z-score normalization: #{}, {}".format(total, file))
            pt = path.split(os.path.sep)[-1]
            newfilename = file.split(".")[0] + "_znormed.nii.gz"
            znorm_outPath = get_outpath(znormedDir, pt, newfilename)
            if znorm_outPath is None:
                continue
            
            seq = find_seq(file)
            if seq == "ADC":
                print("ADC, copying file...")
                img_nii = nib.load(os.path.join(path, file))
                newfilename = file.split(".")[0] + ".nii.gz"
                nib.save(img_nii, os.path.join(znormedDir, pt, newfilename))
                continue
            img_nii = nib.load(os.path.join(path, file))
            img_data = img_nii.get_data()
            mean, std = np.mean(img_data), np.std(img_data)
            normalized = nib.Nifti1Image((img_data - mean) / std, img_nii.affine, img_nii.header)
            nib.save(normalized, znorm_outPath)
        except Exception as e:
            print(path, file)
            print(e)

Z-score normalization: #1, 8355900_T2_registered.nii.gz


/home/sc-mlsg/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:22: DeprecationWarning: get_data() is deprecated in favor of get_fdata(), which has a more predictable return type. To obtain get_data() behavior going forward, use numpy.asanyarray(img.dataobj).

* deprecated from version: 3.0
* Will raise <class 'nibabel.deprecator.ExpiredDeprecationError'> as of version: 5.0


Z-score normalization: #2, 8355900_T1C_registered.nii.gz
Z-score normalization: #3, 8704204_T1C_registered.nii.gz
Z-score normalization: #4, 8704204_T2_registered.nii.gz
Z-score normalization: #5, 8445407_T1C_registered.nii.gz
Z-score normalization: #6, 8445407_T2_registered.nii.gz
Z-score normalization: #7, 5329806_T1C_registered.nii.gz
Z-score normalization: #8, 5329806_T2_registered.nii.gz
Z-score normalization: #9, 5165936_T1C_registered.nii.gz
Z-score normalization: #10, 5165936_T2_registered.nii.gz
Z-score normalization: #11, 4315438_T1C_registered.nii.gz
Z-score normalization: #12, 4315438_T2_registered.nii.gz
Z-score normalization: #13, 10088264_T2_registered.nii.gz
Z-score normalization: #14, 10088264_T1C_registered.nii.gz
Z-score normalization: #15, 5764412_T1C_registered.nii.gz
Z-score normalization: #16, 5764412_T2_registered.nii.gz
Z-score normalization: #17, 5584298_T1C_registered.nii.gz
Z-score normalization: #18, 5584298_T2_registered.nii.gz
Z-score normalization: #19, 

## below are previous codes

In [ ]:
srcDir = 
    print("N4 bias correction: ", pt)
    correctionDir = os.path.join(targetDir, "correction")
    
    if not os.path.exists(correctionDir):
        os.makedirs(correctionDir)
                          
    for file in os.listdir(resampleDir):
        filePath = os.path.join(resampleDir, pt, file)
        
        if not os.path.exists(os.path.join(correctionDir, pt)):
            os.makedirs(os.path.join(correctionDir, pt))
            
        outPath = os.path.join(correctionDir, pt, file)
        #!ResampleImage 3 {filePath} {outPath} 1x1x1
        print(filePath, outPath)

In [ ]:

for (BB, T2, WB) in zip(T1_BB, T2, T1_WB):
    # patient ID number (MRN) is extracted as a variable 'mrn'
    mrn = BB.split('_')[0] 
    print((BB, T2, WB))
     print("<<< Started working on the following case: ", mrn, " >>>")
        
    # 0. Copy the current T1 and T2 files onto the working directory 
    #    (to avoid 'sudo' problem in trying to write a file on an external storage)
    pathList = []
    for file in os.listdir(patientDir):
        shutil.copyfile(os.path.join(patientDir, file), os.path.join(workingDir, file))
        pathList.append(os.path.join(workingDir, file))
    
    # 1. Resampling onto 1mm x 1mm x 1mm images 
    print("1) Resampling to 1-mm isovoxel images: ", end="")
    for file in pathList:
        !ResampleImage 3 {file} {file} 1x1x1
        print(file, end="  ")

In [ ]:
######################
##########metastasis for PYW ###########3


import os
import shutil

# we'll work in the MONSTR directory
workingDir = '/home/chansik/MONSTR'
%cd {workingDir} 

# set paths for source and target
sourceDir = '/media/chansik/Samsung_T5/mets_nifti'
targetDir = '/media/chansik/Samsung_T5/mets_stripped'
outputPath = '/home/chansik/MONSTR/output'
##########################################################################################

alreadyDoneList = []

# file names MUST be BB.nii, WB.nii, T2.nii, FLAIR.nii, and ADC.nii

for patient in os.listdir(sourceDir):
    patientDir = os.path.join(sourceDir, patient)
    
    # check if preprocessing has already been done
    # if so, skip the patient and move onto the next patient
    if patient in os.listdir(targetDir):
        print(patient, 'has already been processed!')
        alreadyDoneList.append(patient)
        continue
    
    print("<<< Started working on the following case: ", patient, " >>>")
    
    # 0. Copy the current T1 and T2 files onto the working directory 
    #    (to avoid 'sudo' problem in trying to write a file on an external storage)
    pathList = []
    for file in os.listdir(patientDir):
        shutil.copyfile(os.path.join(patientDir, file), os.path.join(workingDir, file))
        pathList.append(os.path.join(workingDir, file))
    
    # 1. Resampling onto 1mm x 1mm x 1mm images 
    print("1) Resampling to 1-mm isovoxel images: ", end="")
    for file in pathList:
        !ResampleImage 3 {file} {file} 1x1x1
        print(file, end="  ")
    
    # 2. N4 bias correction
    # by using ANTs, using the same parameters as the default one used by MONSTR
    print("\n2) N4 bias field correction: ", end="")
    for file in pathList:
        !N4BiasFieldCorrection -d 3 -i {file} -o {file} -s 3 -c [50x50x50x50,0.00001] -b [150]
        print(file, end="  ")
  
    # 3. Co-registration
    # by using ANTs, using the same parameters as the default one used by MONSTR
    print("\n3) Co-registration with CE-T1 images as fixed, reference images: ", end="")
    for file in ["T2.nii", "WB.nii"]:
        newName = file.split(".")[0] + "_registered.nii"
        !antsRegistration -d 3 -r [BB.nii, {file}, 1] \
                          -m mattes[BB.nii, {file}, 1, 32, regular, 0.1 ] \
                          -t translation[0.1] -c [100x50x25,1.e-8,20] -s 4x2x1vox -f 6x4x2 -l 1 \
                          -m mattes[BB.nii, {file} , 1 , 32, regular, 0.1] \
                          -t rigid[0.1] -c [100x50x25,1.e-8,20]  -s 4x2x1vox  -f 3x2x1 -l 1 -o [reg, {newName}] -v 0
        print(file, end="  ")
    shutil.copyfile("BB.nii", "BB_registered.nii")
 
    # 4. MONSTR skull stripping
    t1 = "WB_registered.nii"
    t2 = "T2_registered.nii"
    !./MONSTR.sh --t1 {t1} --t2 {t2} --o output --atlasdir TBI_atlas --reg

    # Copy the relevant files (brain Mask and processed images)
    createdDir = os.path.join(outputPath, os.listdir(outputPath)[0])

    os.makedirs(os.path.join(targetDir, patient))
    for file in os.listdir(createdDir):
        if file == 'atlas4_brainmask_ANTS.nii.gz':
            newFileName = patient + '/brain_mask.nii.gz'
            shutil.copyfile(os.path.join(createdDir, file), os.path.join(targetDir, newFileName))

    shutil.copyfile("WB_registered.nii", os.path.join(targetDir, patient, "WB_registered.nii"))
    shutil.copyfile("BB_registered.nii", os.path.join(targetDir, patient, "BB_registered.nii"))
    shutil.copyfile("T2_registered.nii", os.path.join(targetDir, patient, "T2_registered.nii"))
 
    # 5. Clean the temporarily created files and directories
    shutil.rmtree(createdDir)

In [ ]:
sourceDir = '/media/chansik/Samsung_T5/test'
targetDir = '/media/chansik/Samsung_T5/mets_stripped'
outputPath = '/home/chansik/MONSTR/output'

# Copy the relevant files (brain Mask and processed images)
createdDir = os.path.join(outputPath, os.listdir(outputPath)[0])
patient = '1234'

os.makedirs(os.path.join(targetDir, patient))
for file in os.listdir(createdDir):
    if file == 'atlas4_brainmask_ANTS.nii.gz':
        newFileName = patient + '/brain_mask.nii.gz'
        shutil.copyfile(os.path.join(createdDir, file), os.path.join(targetDir, newFileName))
    
shutil.copyfile("WB_registered.nii", os.path.join(targetDir, patient, "WB_registered.nii"))
shutil.copyfile("BB_registered.nii", os.path.join(targetDir, patient, "BB_registered.nii"))
shutil.copyfile("T2_registered.nii", os.path.join(targetDir, patient, "T2_registered.nii"))

# Clean the temporarily created files and directories
#os.remove(os.path.join(workingDir, t1))
#os.remove(os.path.join(workingDir, t2))
#shutil.rmtree(createdDir)


In [ ]:
t1 = "BB_registered.nii"
t2 = "T2_registered.nii"

!./MONSTR.sh --t1 {t1} --t2 {t2} --o output --atlasdir TBI_atlas --reg

In [ ]:
 !./MONSTR.sh --t1 {t1} --t2 {t2} --o output --atlasdir TBI_atlas --reg

    # 3. Copy the relevant files (brain Mask and processed T1/T2 images)
    createdDir = os.path.join(outputPath, os.listdir(outputPath)[0])

    for file in os.listdir(createdDir):
        if file == 'atlas4_brainmask_ANTS.nii.gz':
            newFileName = mrn + '_bMask.nii.gz'
            shutil.copyfile(os.path.join(createdDir, file), os.path.join(targetDir, newFileName))

        if file == 't1.nii.gz':
            newFileName = mrn + '_T1GD_processed.nii.gz'
            shutil.copyfile(os.path.join(createdDir, file), os.path.join(targetDir, newFileName))

        if file == 't2.nii.gz':
            newFileName = mrn + '_T2_processed.nii.gz'
            shutil.copyfile(os.path.join(createdDir, file), os.path.join(targetDir, newFileName))

    # 4. Clean the temporarily created files and directories
    os.remove(os.path.join(workingDir, t1))
    os.remove(os.path.join(workingDir, t2))
    shutil.rmtree(createdDir)

In [ ]:
######################
##########test ###########3


import os
import shutil

# we'll work in the MONSTR directory
workingDir = '/home/chansik/MONSTR'
%cd {workingDir} 

# set paths for source and target
sourceDir = '/media/chansik/Samsung_T5'
targetDir = '/media/chansik/SC_research_2019/GBM-Surv/Seg_results'
##########################################################################################

alreadyDoneList = []

# file names MUST be CE-T1.nii, UE-T1.nii, T2.nii, FLAIR.nii, and ADC.nii

for patient in os.listdir(sourceDir):
    patientDir = os.path.join(sourceDir, patient)
    
    # check if preprocessing has already been done
    # if so, skip the patient and move onto the next patient
    if patient in os.listdir(targetDir):
        print(patient, 'has already been processed!')
        alreadyDoneList.append(patient)
        continue
    
    print("<<< Started working on the following case: ", patient, " >>>")
    
    # 0. Copy the current T1 and T2 files onto the working directory 
    #    (to avoid 'sudo' problem in trying to write a file on an external storage)
    pathList = []
    for file in os.listdir(patientDir):
        shutil.copyfile(os.path.join(patientDir, file), os.path.join(workingDir, file))
        pathList.append(os.path.join(workingDir, file))
    
    # 1. Resampling onto 1mm x 1mm x 1mm images 
    print("1) Resampling to 1-mm isovoxel images: ", end="")
    for file in pathList:
        !ResampleImage 3 {file} {file} 1x1x1
        print(file, end="  ")
    
    # 2. N4 bias correction
    # by using ANTs, using the same parameters as the default one used by MONSTR
    print("\n2) N4 bias field correction: ", end="")
    for file in pathList:
        !N4BiasFieldCorrection -d 3 -i {file} -o {file} -s 3 -c [50x50x50x50,0.00001] -b [150]
        print(file, end="  ")
  
    # 3. Co-registration
    # by using ANTs, using the same parameters as the default one used by MONSTR
    print("\n3) Co-registration with CE-T1 images as fixed, reference images: ", end="")
    for file in ["UE-T1.nii", "T2.nii", "FLAIR.nii", "ADC.nii"]:
        newName = file.split(".")[0] + "_registered.nii"
        !antsRegistration -d 3 -r [CE-T1.nii, {file}, 1] \
                          -m mattes[CE-T1.nii, {file}, 1, 32, regular, 0.1 ] \
                          -t translation[0.1] -c [100x50x25,1.e-8,20] -s 4x2x1vox -f 6x4x2 -l 1 \
                          -m mattes[CE-T1.nii, {file} , 1 , 32, regular, 0.1] \
                          -t rigid[0.1] -c [100x50x25,1.e-8,20]  -s 4x2x1vox  -f 3x2x1 -l 1 -o [reg, {newName}] -v 0
        print(file, end="  ")
    shutil.copyfile("CE-T1.nii", "CE-T1_registered.nii")
 

In [ ]:
    # check if preprocessing has already been done
    # if so, skip the patient and move onto the next patient
    if mrn in [i.split('_', 1)[0] for i in os.listdir(targetDir)]:
        print(mrn, 'has already been processed!')
        alreadyDoneList.append(mrn)
        continue
       
    # 0. Copy the current T1 and T2 files onto the working directory 
    #    (to avoid 'sudo' problem in trying to write a file on an external storage)
    shutil.copyfile(os.path.join(sourceDir, t1), os.path.join(workingDir, t1))
    shutil.copyfile(os.path.join(sourceDir, t2), os.path.join(workingDir, t2))
    
    # 1. Resampling of CE-T1-weighted images onto 1mm x 1mm x 1mm images 
    #   (as some T1 images are 2D)
    !ResampleImage 3 {t1} {t1} 1x1x1 
    
    # 2. Skull stripping by MONSTR algorithm following 
    #          1) rigid coregistration of T2 onto T1 (ANTs)
    #          2) N4 bias correction (ANTs)
    !./MONSTR.sh --t1 {t1} --t2 {t2} --o output --atlasdir TBI_atlas 

    # 3. Copy the relevant files (brain Mask and processed T1/T2 images)
    createdDir = os.path.join(outputPath, os.listdir(outputPath)[0])

    for file in os.listdir(createdDir):
        if file == 'atlas4_brainmask_ANTS.nii.gz':
            newFileName = mrn + '_bMask.nii.gz'
            shutil.copyfile(os.path.join(createdDir, file), os.path.join(targetDir, newFileName))

        if file == 't1.nii.gz':
            newFileName = mrn + '_T1GD_processed.nii.gz'
            shutil.copyfile(os.path.join(createdDir, file), os.path.join(targetDir, newFileName))

        if file == 't2.nii.gz':
            newFileName = mrn + '_T2_processed.nii.gz'
            shutil.copyfile(os.path.join(createdDir, file), os.path.join(targetDir, newFileName))

    # 4. Clean the temporarily created files and directories
    os.remove(os.path.join(workingDir, t1))
    os.remove(os.path.join(workingDir, t2))
    shutil.rmtree(createdDir)
    

In [ ]:
workingDir = '/home/chansik/MONSTR'
%cd {workingDir} 
shutil.copyfile("CE-T1.nii", "CE-T1_registered.nii")

In [ ]:
workingDir = '/home/chansik/MONSTR'
%cd {workingDir} 

!./MONSTR.sh --t1 CE-T1_registered.nii --t2 T2_registered.nii --fl FLAIR_registered.nii --o output --atlasdir TBI_atlas --reg --clean


In [ ]:
import numpy as np
import os
import nibabel as nib
#import matplotlib.pyplot as plt
#from matplotlib.image import imsave
#from PIL import Image

workingDir = '/home/chansik/MONSTR'
%cd {workingDir} 

mask_array = nib.load("atlas4_brainmask_ANTS.nii.gz").get_fdata()
    
for file in ["UE-T1_registered.nii", "CE-T1_registered.nii", "T2_registered.nii", "FLAIR_registered.nii", "ADC_registered.nii"]:
    img_array = nib.load(file).get_fdata()
    stripped_array = img_array * mask_array
    
    newName = file.split(".")[0].split("_")[0] + "_stripped.nii"
    
    niftiFile = nib.Nifti1Image(stripped_array, np.eye(4)) 
    nib.save(niftiFile, newName)

In [ ]:
file = "UE-T1_registered.nii"
file.split(".")[0].split("_")[0]

In [ ]:
    
    
    # 2. Skull stripping by MONSTR algorithm following 
    #          1) rigid coregistration of T2 onto T1 (ANTs)
    #          2) N4 bias correction (ANTs)
    !./MONSTR.sh --t1 {t1} --t2 {t2} --o output --atlasdir TBI_atlas 

    # 3. Copy the relevant files (brain Mask and processed T1/T2 images)
    createdDir = os.path.join(outputPath, os.listdir(outputPath)[0])

    for file in os.listdir(createdDir):
        if file == 'atlas4_brainmask_ANTS.nii.gz':
            newFileName = mrn + '_bMask.nii.gz'
            shutil.copyfile(os.path.join(createdDir, file), os.path.join(targetDir, newFileName))

        if file == 't1.nii.gz':
            newFileName = mrn + '_T1GD_processed.nii.gz'
            shutil.copyfile(os.path.join(createdDir, file), os.path.join(targetDir, newFileName))

        if file == 't2.nii.gz':
            newFileName = mrn + '_T2_processed.nii.gz'
            shutil.copyfile(os.path.join(createdDir, file), os.path.join(targetDir, newFileName))

    # 4. Clean the temporarily created files and directories
    os.remove(os.path.join(workingDir, t1))
    os.remove(os.path.join(workingDir, t2))
    shutil.rmtree(createdDir)

In [ ]:
  # 4. MONSTR skull stripping
    t1 = "WB_registered.nii"
    t2 = "T2_registered.nii"
    

    # Copy the relevant files (brain Mask and processed images)
    

    shutil.copyfile("WB_registered.nii", os.path.join(targetDir, patient, "WB_registered.nii"))
    shutil.copyfile("BB_registered.nii", os.path.join(targetDir, patient, "BB_registered.nii"))
    shutil.copyfile("T2_registered.nii", os.path.join(targetDir, patient, "T2_registered.nii"))
 
    # 5. Clean the temporarily created files and directories
    
    

## centering, cropping, and selecting slices

In [ ]:
# define a function

def center_crop(img, margin=10):
    # convert an image into an sqaure image if the dimensions of x and y are different
    if img.shape[0] != img.shape[1]:
        if img.shape[0] < img.shape[1]:
            pad_width = int((img.shape[1] - img.shape[0])//2)
            img = np.pad(img, ((pad_width, pad_width), (0,0), (0,0)), constant_values=0)

        if img.shape[0] > img.shape[1]:
            pad_width = int((img.shape[0] - img.shape[1])//2)
            img = np.pad(img, ((0,0), (pad_width, pad_width), (0,0)), constant_values=0)
        
    # calculate the non-zero margins of x and y axes.
    x_min, x_max = 1000, 0
    y_min, y_max = 1000, 0

    for z in range(img.shape[2]):
        for row in range(img.shape[0]):
            x = np.nonzero(img[row, :, z])

            if x[0].size == 0:
                continue

            if x_min > x[0].min():
                x_min = x[0].min()
            if x_max < x[0].max():
                x_max = x[0].max()

        for col in range(img.shape[1]):
            y = np.nonzero(img[:, col, z])

            if y[0].size == 0:
                continue

            if y_min > y[0].min():
                y_min = y[0].min()
            if y_max < y[0].max():
                y_max = y[0].max()

    x_width = x_max - x_min
    x_center = x_min + (x_width//2)
    y_width = y_max - y_min
    y_center = y_min + (y_width//2)

    if x_width >= y_width:
        new_width = x_width + margin
    else: 
        new_width = y_width + margin

    x_start, x_end = x_center-(new_width//2), x_center+(new_width//2)
    y_start, y_end = y_center-(new_width//2), y_center+(new_width//2)

    new_img = img[y_start:y_end, x_start:x_end, :]
    
    return new_img

In [ ]:
import pandas as pd
import numpy as np
import os
import nibabel as nib
import matplotlib.pyplot as plt
from matplotlib.image import imsave
from PIL import Image

sourceDir = '/media/chansik/SC_research_2019/GBM-Surv/ProcessedByMONSTR'
outputDir = '/home/chansik/DataAnalysis/201912-GBM-Surv/output-multiple'

range_df = pd.read_csv('/home/chansik/DataAnalysis/201912-GBM-Surv/input/range_list.csv', header=None)

target_img_size = (224, 224)

for idx, row in range_df.iterrows():
    t1_name = os.path.join(sourceDir, (str(row[0]) + "_T1GD_processed.nii.gz"))
    t2_name = os.path.join(sourceDir, (str(row[0]) + "_T2_processed.nii.gz"))
    mask_name = os.path.join(sourceDir, (str(row[0]) + "_bMask.nii.gz"))
    
    # Get Image Array (numpy array) from Nifti File
    t1_array = nib.load(t1_name).get_fdata()
    t2_array = nib.load(t2_name).get_fdata()
    mask_array = nib.load(mask_name).get_fdata()
    
    t1_img = t1_array * mask_array
    t1_img = t1_img[:, :, int(row[1]):int(row[2])]
    
    t2_img = t2_array * mask_array
    t2_img = t2_img[:, :, int(row[1]):int(row[2])]
    
    cropped_t1 = center_crop(t1_img, margin=10) # the function 'center_crop' was defined above
    cropped_t2 = center_crop(t2_img, margin=10)
    
    height = int(row[2]) - int(row[1])
    
    for slc in range(2, height, 5):
        selected_t1 = cropped_t1[:, :, slc]
        selected_t2 = cropped_t2[:, :, slc]
        
        newT1name = os.path.join(outputDir, (str(row[0]) + "_T1_" + str(slc) + ".png"))
        newT2name = os.path.join(outputDir, (str(row[0]) + "_T2_" + str(slc) + ".png"))

        imsave(newT1name, selected_t1, cmap='gray')
        imsave(newT2name, selected_t2, cmap='gray')

## resizing and stacking

In [ ]:
import pandas as pd
import numpy as np
import os
import shutil
import random

baseDir = '/home/chansik/DataAnalysis/201912-GBM-Surv'
sourceDir = os.path.join(baseDir, 'output-multiple')
targetDir = os.path.join(baseDir, 'cnn_input')

df = pd.read_csv(os.path.join(baseDir, 'input/GBM-surv-list.csv'))
df = df.loc[df.death==1, ['ID', 'survival']]

short = df.loc[df.survival < df.survival.median(), 'ID'].tolist()
long = df.loc[df.survival >= df.survival.median(), 'ID'].tolist()

short_validation_list = random.choices(short, k=30)
long_validation_list = random.choices(long, k=30)

for file in os.listdir(sourceDir):
    
    # patient ID number (MRN) is extracted as a variable 'mrn'
    mrn = int(file.split('_', 1)[0])
    
    if mrn in short:
        if mrn in short_validation_list:
            shutil.copyfile(os.path.join(sourceDir, file), 
                            os.path.join(targetDir, 'validation/short', file))
        else:
             shutil.copyfile(os.path.join(sourceDir, file), 
                            os.path.join(targetDir, 'train/short', file))
    
    if mrn in long:
        if mrn in long_validation_list:
            shutil.copyfile(os.path.join(sourceDir, file), 
                            os.path.join(targetDir, 'validation/long', file))
        else:
             shutil.copyfile(os.path.join(sourceDir, file), 
                            os.path.join(targetDir, 'train/long', file))

In [ ]:
for file in os.listdir(sourceDir):
    
    # patient ID number (MRN) is extracted as a variable 'mrn'
    mrn = int(file.split('_', 1)[0])
    
    if mrn in df.ID.tolist():
        survival = df.loc[df.ID==mrn, 'survival'].values[0]
        new_fileName = str(survival) + "_" + str(mrn) + ".png"

        if mrn in validation_set:
            shutil.copyfile(os.path.join(sourceDir, file), 
                            os.path.join(targetDir, 'validation', new_fileName))
        else:
            shutil.copyfile(os.path.join(sourceDir, file), 
                            os.path.join(targetDir, 'train', new_fileName))
    

In [ ]:
baseDir = '/home/chansik/DataAnalysis/201912-GBM-Surv'
sourceDir = os.path.join(baseDir, 'output')
targetDir = os.path.join(baseDir, 'cnn_regression')

[p.split('_')[os.listdir(targetDir)

### divide patients into two: short and long

In [ ]:

import pandas as pd
import numpy as np
import os
import shutil

df = pd.read_csv('/home/chansik/DataAnalysis/201912-GBM-Surv/input/GBM-surv-list.csv')
df = df.loc[df.death==1, ['ID', 'survival']]

short = df.loc[df.survival < df.survival.median(), 'ID'].tolist()
long = df.loc[df.survival >= df.survival.median(), 'ID'].tolist()

sourceDir = '/home/chansik/DataAnalysis/201912-GBM-Surv/output'
shortDir = '/home/chansik/DataAnalysis/201912-GBM-Surv/cnn_input/short'
longDir = '/home/chansik/DataAnalysis/201912-GBM-Surv/cnn_input/long'

for file in os.listdir(sourceDir):
    
    # patient ID number (MRN) is extracted as a variable 'mrn'
    mrn = int(file.split('_', 1)[0])

    if mrn in short:
        shutil.copyfile(os.path.join(sourceDir, file), os.path.join(shortDir, file))
    if mrn in long:
        shutil.copyfile(os.path.join(sourceDir, file), os.path.join(longDir, file))

## convert gray to RGB

In [ ]:
import pandas as pd
import numpy as np
import os
import shutil
import tensorflow as tf
from matplotlib.image import imsave

baseDir = '/home/chansik/DataAnalysis/201912-GBM-Surv'
print(os.path.join(baseDir, 'output'))
for file in os.listdir(os.path.join(baseDir, 'output')):
    final_img_name = os.path.join(baseDir, 'gray-to-rgb', file)
    converted = tf.image.grayscale_to_rgb(file, name=None)
    imsave(final_img_name, converted)
    


In [ ]:
from PIL import Image
resized_image = Image.fromarray(new_img[:,:,20]).resize((224, 224))

nifti_file = nib.Nifti1Image(new_img, np.eye(4))
nib.save(nifti_file, 'test.nii') # Save

In [ ]:
plt.imshow(np.array(resized_image))

In [ ]:
plt.imshow(img[:, :, 29])

In [ ]:
print(img.shape)
new = np.pad(img, pad_width = ((1,1), (0,0), (0,0)), constant_values=0)
print(new.shape)

In [ ]:
import pandas as pd
import numpy as np
import os
import nibabel as nib
import matplotlib.pyplot as plt
from matplotlib.image import imsave
from PIL import Image

image_array = nib.load("251020_T1GD_processed.nii.gz").get_fdata()
mask_array = nib.load("251020_bMask.nii.gz").get_fdata()
img = image_array * mask_array
img = img[:, :, 53:113]
 